## Importation librairie et donnée


In [2559]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import re
import os
import time
import matplotlib.pyplot as plt

In [2560]:
pd.set_option('display.max_columns', 150)

In [2561]:
def read_and_process_file(fichier_a_deposee):
    vol_prevu = []
    vol_fini = []
    vol_termine = []
    tableau_vol = {}
    tableaux_vol = []
    iter = 0
    flag82 = False
    hneg = False
    compt82 = 0
    num81 = 0
    prevu = False
    termine = False
    final = False
    complet = 0
    output = pd.DataFrame()
    compteur = 0
    tableau_vol = {}
    compteurCcr = 0

    with open(fichier_a_deposee, 'r') as fichier:
        for i, ligne in enumerate(fichier):
            words = ligne.split()
            if words[0] == "02":
                date_str = words[1]
                date_obj = datetime.strptime(date_str, "%d-%m-%Y")
                print('date obj ',date_obj)
                date_fichier = date_obj.timetuple().tm_yday
                print('date fichier ',date_fichier)
            if words[0] == "05":
                if tableau_vol:
                    tableau_vol["isPrevu"] = isprevu
                    tableau_vol["isRealise"] = isrealise
                    tableau_vol["isFinal"] = isfinal
                    df_dictionary = pd.DataFrame([tableau_vol])
                    output = pd.concat([output, df_dictionary], ignore_index=True)
                tableau_vol = {}
                isprevu = False
                isrealise = False
                isfinal = False
                is81 = False
                is82 = False
                balise = ''
                hneg = False
                flag82 = False
                etat = ''
            if words[0] == "11":
                etat = 'prevu'
                isprevu = True
            if words[0] == "12":
                etat = 'realise'
                isrealise = True
            if words[0] == "13":
                if len(words) > 1 and words[1] == "=":
                    for key in list(tableau_vol.keys()):
                        if '_prevu' in key:
                            tableau_vol[key.replace('_prevu', '_final')] = tableau_vol[key]
                etat = 'final'
                isfinal = True
            if words[0] == "14":
                etat = 'transaction'
            if words[0] == "81":
                pass
            if words[0] == "20":
                tableau_vol['callSign_' + etat] = words[1]
                tableau_vol['dep_' + etat] = words[2]
                tableau_vol['arr_' + etat] = words[3]
                tableau_vol['numCautra_' + etat] = words[4]
                tableau_vol['dateRelative_' + etat] = words[5]
                tableau_vol['typeAvion_' + etat] = words[6]
                tableau_vol['work_' + etat] = words[7]
                if words[8][:2] == '??':
                    pass
                else:
                    tableau_vol['work1' + etat] = words[8].strip().ljust(9)
            if words[0] == "21":
                tableau_vol['heuresDep_' + etat] = words[1]
                tableau_vol['RFL_' + etat] = words[2]
                tableau_vol['vitesse_' + etat] = words[3]
                tableau_vol['EOBT_' + etat] = words[4]
            if words[0] == "22":
                tableau_vol['regleVol_' + etat] = words[1]
                tableau_vol['typeVol_' + etat] = words[2]
                tableau_vol['HeurePremiereBaliseActive_' + etat] = words[10]
                if words[3][:2] == '??':
                    pass
                else:
                    tableau_vol['IFPL_' + etat] = words[3].strip().ljust(10)
                tableau_vol['plnActive_' + etat] = words[4]
                tableau_vol['typePlnStan']= words[6]
                tableau_vol['plnAnnule_' + etat] = words[5]
                if '??' in words[7]:
                    pass
                elif len(words[7]) == 8:
                    date_str = words[7]
                    date_obj = datetime.strptime(date_str, '%d%m%Y')
                    day_vol = date_obj.timetuple().tm_yday
                    tableau_vol['dateBlock_' + etat] = words[7][:4] + words[7][6:]
                else:
                    tableau_vol['dateBlock_' + etat] = words[7].strip().ljust(6)
            if words[0] == "23":
                if "??" in words[4]:
                    tableau_vol['adresseModeS_' + etat] = np.NaN
                else:
                    tableau_vol['adresseModeS_' + etat] = words[4]
            if words[0] == "24":
                tableau_vol['numeroPLNM' + etat] = words[1]
                tableau_vol['flightID' + etat] = words[2]
            if words[0] == "31":
                tableau_vol['balise' + etat] = words[1]
            if words[0] == "32":
                tableau_vol['HeurePremiereBalise_' + etat] = words[1]
            if words[0] == "33":
                tableau_vol['listeBalises' + etat] = words[1]
            if words[0] == "36":
                tableau_vol['indicateur' + etat] = words[1]
            if words[0] == "41":
                tableau_vol['carte' + etat] = words[1]
            if words[0] == "71":
                tableau_vol['centreTraversé' + etat] = words[1:]
            if words[0] == "72":
                tableau_vol['listeRangPremier' + etat] = words[1]
            if words[0] == "80":
                tableau_vol['rangTransaction' + etat] = words[1]
            if words[0] == "81" and not is81:
                is81 = True
                if len(words) >= 15:
                    parts = ligne.split("-")
                    last_word = parts[0].split()[-1]
                    if "ABI" in ligne:
                        tableau_vol['typePln'] = "ABI"
                    if "(FPL" in parts[0] or "(CHG)" in parts[0]:
                        tableau_vol['case7'] = parts[1]
                        tableau_vol['case8'] = parts[2]
                        tableau_vol['case9'] = parts[3]
                        tableau_vol['case10'] = parts[4]
                        tableau_vol['case13'] = parts[5]
                        tableau_vol['case15'] = parts[6]
                        if len(parts) > 8:
                            tableau_vol['case16'] = parts[7]
                            tableau_vol['case18'] = parts[8]
                            if tableau_vol['case18'] == "RPL":
                                tableau_vol['typePln'] = "RPL"
                        else:
                            #print(ligne)
                            compteur += 1
                    elif "(APL" in parts[0]:
                        tableau_vol['case7'] = parts[1]
                        tableau_vol['case8'] = parts[2]
                        tableau_vol['case9'] = parts[3]
                        tableau_vol['case10'] = parts[4]
                        tableau_vol['case13'] = parts[5]
                        tableau_vol['case15'] = parts[6]
                        tableau_vol['typePln'] = "APL"
                        if len(parts) > 8:
                            tableau_vol['case16'] = parts[7]
                            tableau_vol['case18'] = parts[-1]
                        else:
                            print(ligne)
                            compteur += 1
            if words[0] == "82"and not is82:
                is82 = True
                tableau_vol['heure'] = (words[1][:2])
                tableau_vol['minute'] = (words[1][3:])
                tableau_vol['accuseTrt' + etat] = words[1]
                if "CCR:" in ligne:
                    compteurCcr = 0
                    for word in words:
                        compteurCcr += 1
                        if word == "CCR:":
                            break
                    tableau_vol['ccrArrival'] = words[compteurCcr]
            if words[0] == "84":
                tableau_vol['final' + etat] = words[1]

    return output

def convert_and_calculate(df,date_obj):
    df['HeurePremiereBaliseActive_realise'] = df['HeurePremiereBaliseActive_realise'].astype('Int64')
    df['HeurePremiereBaliseActive_final'] = df['HeurePremiereBaliseActive_final'].astype('Int64')
    df['HeurePremiereBalise_final'] = df['HeurePremiereBalise_final'].astype('Int64')
    df['dateRelative_realise'] = df['dateRelative_realise'].astype('Int64')
    df['dateRelative_final'] = df['dateRelative_final'].astype('Int64')

    def calcul_HeureDeReference(row):
        try:
            if not pd.isna(row['dateRelative_realise']) and not pd.isnull(row['dateRelative_realise']):
                if not pd.isna(row['HeurePremiereBaliseActive_realise']) and int(row['HeurePremiereBaliseActive_realise']) != 0:
                    return int(row['HeurePremiereBaliseActive_realise']) + (-1440 if row['dateRelative_realise'] == -1 else 1440 if row['dateRelative_realise'] == 1 else 0)
                elif not pd.isna(row['HeurePremiereBalise_final']):
                    return int(row['HeurePremiereBalise_final']) + (-1440 if row['dateRelative_realise'] == -1 else 1440 if row['dateRelative_realise'] == 1 else 0)
            elif not pd.isna(row['dateRelative_final']):
                if not pd.isna(row['HeurePremiereBaliseActive_final']) and int(row['HeurePremiereBaliseActive_final']) != 0:
                    return int(row['HeurePremiereBaliseActive_final']) + (-1440 if row['dateRelative_final'] == -1 else 1440 if row['dateRelative_final'] == 1 else 0)
                elif not pd.isna(row['HeurePremiereBalise_final']):
                    return int(row['HeurePremiereBalise_final']) + (-1440 if row['dateRelative_final'] == -1 else 1440 if row['dateRelative_final'] == 1 else 0)
        except Exception:
            return None

    df['heure_de_reference'] = df.apply(calcul_HeureDeReference, axis=1)

    def calcul_DateDeReference(row):
        print('date obj func date ref',date_obj)

        try:
            if not pd.isna(row['dateRelative_realise']) and not pd.isnull(row['dateRelative_realise']):
                if row['dateRelative_realise'] == 0:
                    return date_obj
                elif row['dateRelative_realise'] == 1:
                    return date_obj + timedelta(days=1)
                elif row['dateRelative_realise'] == -1 and int(row['heure_de_reference'])<0:
                    return date_obj - timedelta(days=1)
                else:
                    return date_obj
            elif not pd.isna(row['dateRelative_final']) and not pd.isnull(row['dateRelative_final']):
                if row['dateRelative_final'] == 0:
                    return date_obj
                elif row['dateRelative_final'] == 1:
                    return date_obj + timedelta(days=1)
                elif row['dateRelative_final'] == -1 and int(row['heure_de_reference'])<0:
                    return date_obj - timedelta(days=1)
                else:
                    return date_obj
        except Exception as e:
            return None

    df['date_de_reference'] = df.apply(calcul_DateDeReference, axis=1)
    return df

In [2562]:
from datetime import datetime, timedelta
import locale

locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

def dynamic_analysis(date_fichier):
    target_date_str = date_fichier
    # Convert the target_date_str to a datetime object
    target_date = datetime.strptime(target_date_str, "%d-%m-%Y")
    print('target date',target_date)
    # Calculate the previous and next days
    previous_day = target_date - timedelta(days=1)
    print('previous_day',previous_day)
    next_day = target_date + timedelta(days=1)
    print('next_day',next_day)
    
    # Format the dates to match the file naming convention
    previous_day_str = previous_day.strftime("%Y%m%d")
    target_date_str = target_date.strftime("%Y%m%d")
    next_day_str = next_day.strftime("%Y%m%d")

    date_target = datetime.strptime(date_fichier, '%d-%m-%Y')
    # Extraire le nom du mois en français
    mois_francais = date_target.strftime('%B').lower()
    annee = date_target.year


    
    # Construct file names
    nom_fichier_jour_1 = f'../data/stan_{mois_francais}_{annee}/RDVC-{previous_day_str}.pln'
    nom_fichier_jour_2 = f'../data/stan_{mois_francais}_{annee}/RDVC-{target_date_str}.pln'
    nom_fichier_jour_3 = f'../data/stan_{mois_francais}_{annee}/RDVC-{next_day_str}.pln'
    print('jour de veille',nom_fichier_jour_1)
    print('jour de jour',nom_fichier_jour_2)
    print('jour de lendemain',nom_fichier_jour_3)
    
    # Read and process files
    fichier_jour_1 = read_and_process_file(nom_fichier_jour_1)
    fichier_jour_2 = read_and_process_file(nom_fichier_jour_2)
    fichier_jour_3 = read_and_process_file(nom_fichier_jour_3)
    
    # Convert and calculate
    date_obj = previous_day
    jour_1 = convert_and_calculate(fichier_jour_1,date_obj)
    print('date obj ',date_obj)

    date_obj = target_date
    jour_2 = convert_and_calculate(fichier_jour_2,date_obj)
    print('date obj ',date_obj)

    date_obj = next_day
    jour_3 = convert_and_calculate(fichier_jour_3,date_obj)
    print('date obj ',date_obj)
    
    # Return the results
    return jour_1, jour_2, jour_3

# Example usage
dateAnalyse='24-01-2024'
jour_1, jour_2, jour_3 = dynamic_analysis(dateAnalyse)

target date 2024-01-24 00:00:00
previous_day 2024-01-23 00:00:00
next_day 2024-01-25 00:00:00
jour de veille ../data/stan_janvier_2024/RDVC-20240123.pln
jour de jour ../data/stan_janvier_2024/RDVC-20240124.pln
jour de lendemain ../data/stan_janvier_2024/RDVC-20240125.pln
date obj  2024-01-23 00:00:00
date fichier  23
date obj  2024-01-24 00:00:00
date fichier  24
date obj  2024-01-25 00:00:00
date fichier  25
date obj func date ref 2024-01-23 00:00:00
date obj func date ref 2024-01-23 00:00:00
date obj func date ref 2024-01-23 00:00:00
date obj func date ref 2024-01-23 00:00:00
date obj func date ref 2024-01-23 00:00:00
date obj func date ref 2024-01-23 00:00:00
date obj func date ref 2024-01-23 00:00:00
date obj func date ref 2024-01-23 00:00:00
date obj func date ref 2024-01-23 00:00:00
date obj func date ref 2024-01-23 00:00:00
date obj func date ref 2024-01-23 00:00:00
date obj func date ref 2024-01-23 00:00:00
date obj func date ref 2024-01-23 00:00:00
date obj func date ref 2024-

C:\Users\latti\AppData\Local\Temp\ipykernel_52804\532306668.py:227: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['date_de_reference'] = df.apply(calcul_DateDeReference, axis=1)
C:\Users\latti\AppData\Local\Temp\ipykernel_52804\532306668.py:227: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['date_de_reference'] = df.apply(calcul_DateDeReference, axis=1)


date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj func date ref 2024-01-25 00:00:00
date obj fu

In [2563]:
from datetime import datetime

date_obj = datetime.strptime(dateAnalyse, '%d-%m-%Y')

mois = f"{date_obj.month:02d}"
annee = date_obj.year

print(f"Mois: {mois}, Année: {annee}")

Mois: 01, Année: 2024


In [2564]:
print(date_obj.strftime('%y%m'))

2401


In [2565]:
nom_abacus= f"LF20LF10%S%ABACUS_FLSPBZ03_{date_obj.strftime('%y%m')}.TXT"
print(nom_abacus)

LF20LF10%S%ABACUS_FLSPBZ03_2401.TXT


In [2566]:
# nom_fichier_jour_1 = '../data/stan_mars_2024/RDVC-20240315.pln'
# nom_fichier_jour_2 = '../data/stan_mars_2024/RDVC-20240316.pln'
# nom_fichier_jour_3 = '../data/stan_mars_2024/RDVC-20240317.pln'
# fichier_jour_1 = read_and_process_file(nom_fichier_jour_1)
# fichier_jour_2 = read_and_process_file(nom_fichier_jour_2)
# fichier_jour_3 = read_and_process_file(nom_fichier_jour_3)

In [2567]:
# date_obj = datetime.strptime('15-03-2024', "%d-%m-%Y")
# jour_1 = convert_and_calculate(fichier_jour_1)
# date_obj = datetime.strptime('16-03-2024', "%d-%m-%Y")
# jour_2 = convert_and_calculate(fichier_jour_2)
# date_obj = datetime.strptime('17-03-2024', "%d-%m-%Y")
# jour_3 = convert_and_calculate(fichier_jour_3)

In [2568]:
jour_1['date_de_reference'].value_counts()

date_de_reference
2024-01-23    5903
2024-01-22     167
2024-01-24       2
Name: count, dtype: int64

# Algo 2.3 et 2.4

## Tables utilisées

In [2569]:
INDICATIF_FICTIF = pd.read_csv("../data/TABLE_SYSTEME/indicatifsfictifs", sep=';', skiprows=3)
EUROPE_SUD = pd.read_csv("../data/TABLE_SYSTEME/payssud", sep=';', skiprows=3)
AERONEFS_DE_MOINS_DE_2_TONNES = pd.read_csv("../data/TABLE_SYSTEME/aeronefsmoin2tonnes", sep=';', skiprows=3)
INDICATEURS_D_EMPLACEMENT_FAUX = pd.read_csv("../data/TABLE_SYSTEME/aerodromesfaux", sep=';', skiprows=3)
AERODROME_A_VERIFIER = pd.read_csv("../data/TABLE_SYSTEME/aerodromesaverifier", sep=';', skiprows=3)
AERODROMES_D_APPROCHE = pd.read_csv("../data/TABLE_SYSTEME/aerodromesapproche", sep=';', skiprows=3)
AERODROMES_FRONTALIERS = pd.read_csv("../data/TABLE_SYSTEME/aerodromesfrontaliers", sep=';', skiprows=3)
INDICATIF_A_CODE_AUTO = pd.read_csv("../data/TABLE_SYSTEME/indicatifscodesauto", sep=';', skiprows=3)
INDICATIFS_A_STRUCTURE_TRIGRAMME_CORRECTE = pd.read_csv("../data/TABLE_SYSTEME/structurtrigrammecorrects", sep=';', skiprows=3)
COMPAGNIES_BIGRAMMES_ET_SUFFIXES = pd.read_csv("../data/TABLE_SYSTEME/cgniebigrammesetsuffixes", sep=';', skiprows=3)
COMPAGNIES_AVEC_TRIGRAMME = pd.read_csv("../data/TABLE_SYSTEME/compagniesavectrigrammesautorise", sep=';', skiprows=3)
INDICATIFS_TRIGRAMME_A_VERIFIER = pd.read_csv("../data/TABLE_SYSTEME/structuretrigrammeverif", sep=';', skiprows=3)
LETTRES_AIR_FRANCE = pd.read_csv("../data/TABLE_SYSTEME/lettresairfrance", sep=';', skiprows=3)
LETTRES_AIR_FRANCE = LETTRES_AIR_FRANCE[~((LETTRES_AIR_FRANCE['Lettre'].str.contains("MODIFIE", na=False)) & (LETTRES_AIR_FRANCE['Code exo/trait'].isna()) & (LETTRES_AIR_FRANCE['Commentaire'].isna()))]
LETTRES_AIR_FRANCE.reset_index(drop=True, inplace=True)
IMMATRICULATION_A_VERIFIER = pd.read_csv("../data/TABLE_SYSTEME/immataverif", sep=';', skiprows=3)
IMMATRICULATION_CORRECTE = pd.read_csv("../data/TABLE_SYSTEME/immatcorrectes", sep=';', skiprows=3)
OPERATEURS_MILITAIRES = pd.read_csv("../data/TABLE_SYSTEME/oprmili", sep=';', skiprows=3)
AERONEFS_STRICTEMENT_MILITAIRES = pd.read_csv("../data/TABLE_SYSTEME/aeronefmilitaire", sep=';', skiprows=3)

In [2570]:
# Fonction pour convertir un modèle en expression régulière
def pattern_to_regex(pattern):
    # Remplacer # par .*, + par [a-zA-Z] et - par [0-9]
    regex = pattern.replace('#', '.?').replace('-', '[0-9]').replace('+', '[a-zA-Z]')
    # Ajouter les délimiteurs de début et de fin
    regex = '^' + regex + '$'
    return regex

def trouver_pattern(x, pattern_list):
    for pattern in pattern_list:
        regex = pattern_to_regex(pattern)
        if re.match(regex, x):
            return True
    return False

def renvoie_pattern(x, pattern_list):
    for pattern in pattern_list:
        regex = pattern_to_regex(pattern)
        if re.match(regex, x):
            return pattern
    return "NULL"

## Prétraitement utile/inutile

In [2571]:
def nbre_centre_francais(list):
    rep = 0
    for x in list:
      if x in ['PARI', 'AIX', 'BRST', 'REIM', 'BORD']:
        rep +=1
    return rep

In [2572]:
def utile_inutile(element):
    # Fonction pour trouver une valeur valide dans les colonnes multiples
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not(element[[col]].isna().iloc[0]):
                return element[col]
        return None

    call_sign_value = get_valid_value(element, ["callSign_realise", "callSign_final", "callSign_prevu"])
    PLN_active_value = get_valid_value(element, ["plnActive_realise", "plnActive_final", "plnActive_prevu"])
    PLN_annule_value = get_valid_value(element, ["plnAnnule_realise", "plnAnnule_final", "plnAnnule_prevu"])
    centre_traverse_value = get_valid_value(element, ["centreTraversérealise", "centreTraverséfinal", "centreTraverséprevu"])
    dep_value = get_valid_value(element, ["dep_realise", "dep_final", "dep_prevu"])
    arr_value = get_valid_value(element, ["arr_realise", "arr_final", "arr_prevu"])

    if trouver_pattern(call_sign_value, INDICATIF_FICTIF["Indicatif"].to_list()):
        return "FICT"
    if PLN_active_value == "1":
        return "UTI"
    elif PLN_annule_value == "1":
        return "CNL"
    elif nbre_centre_francais(centre_traverse_value) >= 2:
        return "2SLF"
    elif dep_value == arr_value:
        return "CIRC"
    elif dep_value[0] == "E" and dep_value in EUROPE_SUD["Code pays"].to_list():
        return "NORD"
    elif (dep_value[0] in EUROPE_SUD["Code pays"].to_list() or dep_value[:2] in EUROPE_SUD) and dep_value[0] == "E":
        return "SUD"
    return "UTI"


In [2573]:
def traitement_utile_inutile(df):
    df['utile_inutile'] = df.apply(lambda x: utile_inutile(x), axis=1)
    return df

In [2574]:
def test_utile_vol_active_non_fictif():
    utilite = result[result["callSign_prevu"]=="TRA79Y"]["utile_inutile"].iloc[0]
    assert utilite == "UTI", "utilité diff de UTI"

def test_inutile_vol_fictif_avec_moins():
    utilite = result[result["callSign_prevu"]=="RFIN65"]["utile_inutile"].iloc[0]
    assert utilite == "FICT", "utilité diff de FICT"

def test_inutile_vol_annule():
    utilite = result[result["callSign_prevu"]=="THY90F"]["utile_inutile"].iloc[0]
    assert utilite == "CNL", "utilité diff de CNL"

def test_inutile_vol_centrefr():
    utilite = result[result["callSign_prevu"]=="TRA4N"]["utile_inutile"].iloc[0]
    assert utilite == "2SLF", "utilité diff de 2SLF"

def test_inutile_vol_circulaire():
    utilite = result[result["callSign_prevu"]=="FNY5118"]["utile_inutile"].iloc[0]
    assert utilite == "CIRC", "utilité diff de CIRC"

def test_utile_vol_tout_les_test_faux():
    utilite = result[result["callSign_prevu"]=="TOM22B"]["utile_inutile"].iloc[0]
    assert utilite == "UTI", "utilité diff de UTI"

def test_realise_pln_nan():
    utilite = result[result["callSign_prevu"]=="THY90F"]["utile_inutile"].iloc[0]
    assert utilite == "CNL", "utilité diff de CNL"

def test_realise_centre_nan():
    utilite = result[result["callSign_prevu"]=="UAL71"]["utile_inutile"].iloc[0]
    assert utilite == "2SLF", "utilité diff de 2SLF"

## Traitement unitaire

## Initialisation

In [2575]:
def TU_init(df_utile):
    df_utile['aeronef_de_moins_de_deux_tonnes'] = False
    df_utile['vol_a_transmettre'] = False
    df_utile['vol_approche'] = False
    df_utile['vol_interieur'] = False
    df_utile['vol_frontalier'] = False
    df_utile['vol_VFR'] = False
    df_utile["code_d_exoneration"] = 'Z'
    df_utile['code_exploitant'] = 'Z'
    df_utile['compagnie_française'] = False
    df_utile['type_d_avion_militaire'] = False
    df_utile['PLN_à_verifier_TU'] = False
    df_utile['PLN_valide'] = True
    df_utile['RAZ_des_invalidites_de_type_TU'] = False
    df_utile['invalidite_TU'] = [[] for _ in range(len(df_utile))]
    df_utile['type_d_indicatif'] = ""
    return df_utile

## Algo 1

In [2576]:
def TU_1(df_utile):
    def TU_1_element(x):
        # Vérification de PLN_active
        for col in ["plnActive_realise", "plnActive_final", "plnActive_prevu"]:
            if col in x and not pd.isna(x[col]):
                if x[col] == "0":
                    x['invalidite_TU'] = x.get('invalidite_TU', []) + ["NACT"]
                break

        # Vérification de typeavion
        for col in ["typeAvion_realise", "typeAvion_final", "typeAvion_prevu"]:
            if col in x and not pd.isna(x[col]):
                if x[col] == "ZZZZ":
                    x['invalidite_TU'] = x.get('invalidite_TU', []) + ["TYPAV"]
                    x['PLN_valide'] = False
                if x[col] in AERONEFS_DE_MOINS_DE_2_TONNES["Type avion"].to_list():
                    x['aeronef_de_moins_de_deux_tonnes'] = True
                break
        return x
    df_utile = df_utile.apply(TU_1_element, axis=1)
    return df_utile

## Algo 2

In [2577]:
# def TU_2(df_utile):
#     def TU_2_element(x):
#         if len(x['depfinal']) != 4:
#             x['invalidite_TU'].append("DEPAR1")
#             x['PLN_valide'] = False
#         elif x['depfinal'] in INDICATEURS_D_EMPLACEMENT_FAUX:
#             x['invalidite_TU'].append("DEPAR2")
#             x['PLN_valide'] = False
#         elif trouver_pattern(x['depfinal'], AERODROME_A_VERIFIER):
#             x['invalidite_TU'].append("DVPAR3")
#             x['PLN_valide'] = False
#         if len(x['arrfinal']) != 4:
#             x['invalidite_TU'].append("ARRIV1")
#             x['PLN_valide'] = False
#         elif x['arrfinal'] in INDICATEURS_D_EMPLACEMENT_FAUX:
#             x['invalidite_TU'].append("ARRIV2")
#             x['PLN_valide'] = False
#         elif trouver_pattern(x['arrfinal'], AERODROME_A_VERIFIER):
#             x['invalidite_TU'].append("AVRIV3")
#             x['PLN_valide'] = False
#     df_utile = df_utile.apply(TU_2_element, axis=1)
#     return df_utile

In [2578]:
def TU_2(df_utile):
    def TU_2_element(x):
        # Vérification des colonnes de départ
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in x and not pd.isna(x[col]):
                if len(x[col]) != 4:
                    x['invalidite_TU'].append("DEPAR1")
                    x['PLN_valide'] = False
                elif trouver_pattern(x[col], INDICATEURS_D_EMPLACEMENT_FAUX["Code terrain"]):
                    x['invalidite_TU'].append("DEPAR2")
                    x['PLN_valide'] = False
                elif trouver_pattern(x[col], AERODROME_A_VERIFIER["Code terrain"]):
                    x['invalidite_TU'].append("DVPAR3")
                    x['PLN_valide'] = False
            break

        # Vérification des colonnes d'arrivée
        for col in ["arr_realise", "arr_final", "arr_prevu"]:
            if col in x and not pd.isna(x[col]):
                if len(x[col]) != 4:
                    x['invalidite_TU'].append("ARRIV1")
                    x['PLN_valide'] = False
                elif trouver_pattern(x[col], INDICATEURS_D_EMPLACEMENT_FAUX["Code terrain"]):
                    x['invalidite_TU'].append("ARRIV2")
                    x['PLN_valide'] = False
                elif trouver_pattern(x[col], AERODROME_A_VERIFIER["Code terrain"]):
                    x['invalidite_TU'].append("AVRIV3")
                    x['PLN_valide'] = False
            break
        return x
    df_utile = df_utile.apply(TU_2_element, axis=1)
    return df_utile

## Algo 3

In [2579]:
def TU_3_bis(df_utile):
    eurocontrol = ['EB','ED','EF','EG','EH','EI','EK','EL','EN','EP','ES','EV','EY','LA','LB','LC','LD','LE','LG','LH','LI','LJ','LK','LM','LO','LP','LQ','LR','LS','LT','LU','LW','LY','LZ','UD','UG','GC']
    def TU_3_bis_element(x):
        dep_value = None
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                dep_value = x[col]
                break
        if dep_value == "ZZZZ":
            x['PLN_valide'] = False
            x['PLN_à_verifier_TU'] = False
            x['invalidite_TU'].extend(["DEPAR2", "TRANS1"])
        elif dep_value[:2] in eurocontrol:
            x['vol_a_transmettre'] = False
        elif x["ccrArrival"] == "ALGR":
            x['vol_a_transmettre'] = True
        elif x["ccrArrival"] == "ETRG":
            if ((x['dep_prevu'][:1]=='D' or x['dep_prevu'][:1]=='F') and 'TABOT' in x['case15']):
                x['vol_a_transmettre'] = True
        else:
            x['vol_a_transmettre'] = False
        return x

    df_utile = df_utile.apply(TU_3_bis_element, axis=1)
    return df_utile

In [2580]:
# def TU_3(df_utile):
#     def TU_3_element(x):
#         if x["depfinal"] == "ZZZZ":
#             x['PLN_valide'] = False
#             x['PLN_à_verifier_TU'] = False
#             x['invalidite_TU'].append("DEPAR2")
#             x['invalidite_TU'].append("TRANS1")
#         else:
#             if x["depfinal"][:2] in liste_origin_vol | x["depfinal"][:2] in liste_origin_vol:
#                 if trouver_code(x["depfinal"], liste_origin_vol) == "A":
#                     x['vol_a_transmettre'] = False
#                 elif trouver_code(x["depfinal"], liste_origin_vol) == "B":
#                     x['vol_a_transmettre'] = True
#                 elif trouver_code(x["depfinal"], liste_origin_vol) == "C":
#                     if x["AVR"] == 1:
#                         x['vol_a_transmettre'] = "ADET"
#                         x["PLN_valide"] = False
#                         x['invalidite_TU'].append("TRANSC")
#                     else:
#                         x['vol_a_transmettre'] = False
#                 elif trouver_code(x["depfinal"], liste_origin_vol) == "D":
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSD")
#                 elif trouver_code(x["depfinal"], liste_origin_vol) == "E":
#                     if x["SVR"] == 1:
#                         x['vol_a_transmettre'] = False
#                     else:
#                         x['vol_a_transmettre'] = "ADET"
#                         x["PLN_valide"] = False
#                         x['invalidite_TU'].append("TRANSE")
#                 else:
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSI")
#             else:
#                 if x["AVR"] == 1:
#                     x['vol_a_transmettre'] = True
#                 else:
#                     if x["balisefinal"] == "PTGEO":
#                         x['vol_a_transmettre'] = "ADET"
#                         x["PLN_valide"] = False
#                         x['invalidite_TU'].append("TRANSP")
#                     else:
#                         x['vol_a_transmettre'] = False
#     df_utile = df_utile.apply(TU_3_element, axis=1)
#     return df_utile

In [2581]:
# def TU_3(df_utile):
#     def TU_3_element(x):
#         dep_value = None
#         balise_value = None

#         # Vérification des colonnes de départ
#         for col in ["deprealise", "depfinal", "depprevu"]:
#             if col in x and not pd.isna(x[col]) and x[col] != "":
#                 dep_value = x[col]
#                 break
#         for col in ["baliserealise", "balisefinal", "baliseprevu"]:
#             if col in x and not pd.isna(x[col]) and x[col] != "":
#                 balise_value = x[col]
#                 break

#         if dep_value == "ZZZZ":
#             x['PLN_valide'] = False
#             x['PLN_à_verifier_TU'] = False
#             x['invalidite_TU'].extend(["DEPAR2", "TRANS1"])
#         elif dep_value:
#             dep_code = trouver_code(dep_value, liste_origin_vol)
#             if dep_code == "A":
#                 x['vol_a_transmettre'] = False
#             elif dep_code == "B":
#                 x['vol_a_transmettre'] = True
#             elif dep_code == "C":
#                 if x["AVR"] == 1:
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSC")
#                 else:
#                     x['vol_a_transmettre'] = False
#             elif dep_code == "D":
#                 x['vol_a_transmettre'] = "ADET"
#                 x["PLN_valide"] = False
#                 x['invalidite_TU'].append("TRANSD")
#             elif dep_code == "E":
#                 if x["SVR"] == 1:
#                     x['vol_a_transmettre'] = False
#                 else:
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSE")
#             else:
#                 x['vol_a_transmettre'] = "ADET"
#                 x["PLN_valide"] = False
#                 x['invalidite_TU'].append("TRANSI")
#         else:
#             if x["AVR"] == 1:
#                 x['vol_a_transmettre'] = True
#             else:
#                 if balise_value == "PTGEO":
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSP")
#                 else:
#                     x['vol_a_transmettre'] = False

#         return x

#     df_utile = df_utile.apply(TU_3_element, axis=1)
#     return df_utile

##  Algo 4

In [2582]:
def TU_4(df_utile):
    def TU_4_element(x):
        x['vol_a_transmettre'] = True
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                dep_value = x[col]
                break
        for col in ["arr_realise", "arr_final", "arr_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                arr_value = x[col]
                break
        for col in ["plnActive_realise", "plnActive_final", "plnActive_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                plnActive = x[col]
                break

        if dep_value in AERODROMES_D_APPROCHE["Code terrain"].to_list():
            x["vol_approche"] = True

        if arr_value[:2] == "LF":
            x["vol_interieur"] = True

        elif  plnActive == 0 and dep_value in AERODROMES_FRONTALIERS["Code terrain"].to_list():
            x["vol_frontalier"] = True

        return x
    df_utile = df_utile.apply(TU_4_element, axis=1)
    return df_utile

In [2583]:
# def TU_4(df_utile):
#     def TU_4_element(x):
#         x['vol_a_transmettre'] = True
#         if x["depfinal"] in AERODROMES_D_APPROCHE:
#             x["vol_approche"] = True
#         if x["arrfinal"][:2] == "LF":
#             x["vol_interieur"] == True
#         elif x["PLN_activefinal"] != 1 & x["depfinal"] in AERODROMES_FRONTALIERS:
#             x["vol_frontalier"] == True
#     df_utile = df_utile.apply(TU_4_element, axis=1)
#     return df_utile

## Algo 5

In [2584]:
def TU_5(df_utile):
    def TU_5_element(x):
        # Vérification des colonnes balise et regleVol
        balise_value = None
        regleVol_value = None
        dep_value = None
        for col in ["baliserealise", "balisefinal", "baliseprevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                balise_value = x[col]
                break
        for col in ["regleVol_realise", "regleVol_final", "regleVol_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                regleVol_value = x[col]
                break
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                dep_value = x[col]
                break
        # Conditions basées sur les valeurs trouvées
        if x["typePln"] == "VFR" and balise_value.startswith('VFR') and regleVol_value == "V":
            x['vol_a_transmettre'] = False
        elif x["typePln"] in ["APL", "FPL", "FIH", "FII"] and balise_value.startswith('VFR'):
            x['vol_a_transmettre'] = False
            x['vol_VFR'] = True
            x["typePln"] = "VFR"
        else:
            if not x.get('vol_VFR', False) and regleVol_value == "V" and x.get('vol_a_transmettre', True):
                x["PLN_valide"] = False
                x['invalidite_TU'].append("TRANS0")
            elif dep_value and dep_value[:2] != "LF" and not x.get('vol_a_transmettre', True) and regleVol_value == "Z":
                x["PLN_valide"] = False
                x['invalidite_TU'].append("TRANS2")
                x["typePln"] = "AFI"
            if not x.get('vol_VFR', False) and any(s.startswith('TRANS') for s in x['invalidite_TU']):
                x['vol_a_transmettre'] = "ADET"
        if x["typePln"] in ["APL", "FPL"] and regleVol_value in ["Y", "Z"] and x.get('vol_a_transmettre', True):
            x["code_d_exoneration"] = "Y"
        return x
    df_utile = df_utile.apply(TU_5_element, axis=1)
    return df_utile


In [2585]:
# def TU_5(df_utile):
#     def TU_5_element(x):
#         if x["typePln"] == "VFR" and x["baliseprevu"].str.startswith('VFR') and x["regleVol_prevu"] == "V":
#             x['vol_a_transmettre'] = False
#         elif (x["typePln"] == "APL" or x["typePln"] == "FPL" or x["typePln"] == "FIH" or x["typePln"] == "FII") and x["baliseprevu"].str.startswith('VFR'):
#             x['vol_a_transmettre'] = False
#             x['vol_VFR'] = True
#             x["typePln"] = "VFR"
#         else:
#             if x['vol_VFR'] == False and x["regleVol_prevu"] == "V" and x['vol_a_transmettre'] == True:
#                 x["PLN_valide"] == False
#                 x['invalidite_TU'].append("TRANS0")
#             elif x["depprevu"][:2] != "LF" and x['vol_a_transmettre'] == False and x["regleVol_prevu"] == "Z":
#                 x["PLN_valide"] == False
#                 x['invalidite_TU'].append("TRANS2")
#                 x["typePln"] = "AFI"
#             if x['vol_VFR'] == False and any(s.startswith('TRANS') for s in x['invalidite_TU']):
#                 x['vol_a_transmettre'] = "ADET"
#         if (x["typePln"] == "APL" or x["typePln"] == "FPL") and ( x["regleVol_prevu"] == "Y" or  x["regleVol_prevu"] == "Z") and x['vol_a_transmettre'] == True:
#             x["code_d_exoneration"] = "Y"
#     df_utile = df_utile.apply(TU_5_element, axis=1)
#     return df_utile

## Algo 6

In [2586]:
def TU_65_element(x):
    # Fonction pour trouver une valeur valide dans les colonnes multiples
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    type_avion_value = get_valid_value(x, ["typeAvion_realise", "typeAvion_final", "typeAvion_prevu"])
    if type_avion_value in AERONEFS_STRICTEMENT_MILITAIRES['Type avion'].tolist():
        x['type_d_avion_militaire'] = True
        x['PLN_valide'] = False
        x['invalidite_TU'].extend(["TYPA19", "INDI19"])
        filtre_avion = AERONEFS_STRICTEMENT_MILITAIRES[AERONEFS_STRICTEMENT_MILITAIRES["Type avion"] == type_avion_value]
        if not pd.isna(filtre_avion["Code exoneration"].iloc[0]):
            x["code_d_exoneration"] = filtre_avion["Code exoneration"].iloc[0]
            x['code_exploitant'] = filtre_avion["Code exploitant"].iloc[0]
        else:
            x['code_exploitant'] = 'Z'
    return x

In [2587]:
def TU_64_element(x):
    # Fonction pour trouver une valeur valide dans les colonnes multiples
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    type_avion_value = get_valid_value(x, ["typeAvion_realise", "typeAvion_final", "typeAvion_prevu"])
    OPR = None
    if not(pd.isna(x["case18"])):
        OPR = trouver_case18("OPR/", x["case18"])
    if not(pd.isna(x["case18"])) and OPR != "NULL" and OPR in OPERATEURS_MILITAIRES['Nom operateur'].tolist():
        operateur_militaire = OPERATEURS_MILITAIRES[OPERATEURS_MILITAIRES["Nom operateur"] == OPR]
        x["code_d_exoneration"] = operateur_militaire["Code exoneration"].iloc[0]
        x['code_exploitant'] = operateur_militaire["Code exploitant"].iloc[0]
    else:
        if type_avion_value in AERONEFS_STRICTEMENT_MILITAIRES['Type avion'].tolist():
            x['type_d_avion_militaire'] = True
            avion_militaire = AERONEFS_STRICTEMENT_MILITAIRES[AERONEFS_STRICTEMENT_MILITAIRES["Type avion"] == type_avion_value]
            if not pd.isna(avion_militaire["Code exoneration"].iloc[0]):
                x["code_d_exoneration"] = avion_militaire["Code exoneration"].iloc[0]
                x['code_exploitant'] = avion_militaire["Code exploitant"].iloc[0]
            else:
                x['code_exploitant'] = 'Z'
        x['PLN_valide'] = False
        x['invalidite_TU'].extend(["EXO19", "INDI19", "OPR19"])
    return x

In [2588]:
def TU_63_element(x):
    # Fonction pour trouver une valeur valide dans les colonnes multiples
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
    if call_sign_value and trouver_pattern(call_sign_value, IMMATRICULATION_CORRECTE["Immatriculation"]):
        if trouver_pattern(call_sign_value, IMMATRICULATION_A_VERIFIER["Immatriculation"]):
            x['PLN_valide'] = False
            x['invalidite_TU'].extend(["IVDIC$", "EXO$", "OPR$"])
        else:
            x['type_d_indicatif'] = "IM"
        x = TU_65_element(x)
    else:
        x = TU_64_element(x)
    return x

In [2589]:
def TU_62_element(x):
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
    if call_sign_value and call_sign_value[:2] in COMPAGNIES_BIGRAMMES_ET_SUFFIXES["Indicatif"].values:
        x['code_exploitant'] = COMPAGNIES_BIGRAMMES_ET_SUFFIXES[COMPAGNIES_BIGRAMMES_ET_SUFFIXES["Indicatif"] == call_sign_value[:2]]["Code exploitant"].iloc[0]
        if x['vol_interieur'] == True:
            x["code_d_exoneration"] = COMPAGNIES_BIGRAMMES_ET_SUFFIXES[COMPAGNIES_BIGRAMMES_ET_SUFFIXES["Indicatif"] == call_sign_value[:2]]["Code exoneration"].iloc[0]
            x['compagnie_française'] = True
            x['type_d_indicatif'] = "BI"
        else:
            x['PLN_valide'] = False
            x['invalidite_TU'].extend(["INDIC%", "EXO%", "OPR%"])
        x = TU_65_element(x)
    return x

In [2590]:
def TU_612_element(x):
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None

    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])

    if call_sign_value and re.search(r'\d[A-Z]$', call_sign_value) and call_sign_value[-1] == -1:
        x['PLN_valide'] = False
        x['invalidite_TU'].append("INDIC6")
    x = TU_65_element(x)
    return x

In [2591]:
def TU_611_element(x):
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
    if call_sign_value and call_sign_value[-1] in LETTRES_AIR_FRANCE['Lettre'].tolist():
        code_air_france = LETTRES_AIR_FRANCE[LETTRES_AIR_FRANCE['Lettre'] == call_sign_value[-1]]["Code exo/trait"].iloc[0]
        if code_air_france == 2:
            x['PLN_valide'] = False
            x['invalidite_TU'].extend(["INDIC8", "EXO8"])
        elif not code_air_france.isdigit():
            x["code_d_exoneration"] = code_air_france
    else:
        x['PLN_valide'] = False
        x['invalidite_TU'].append("EXO10")
    x = TU_65_element(x)
    return x

In [2592]:
def TU_61_element(x):
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
    if call_sign_value[:3] in COMPAGNIES_AVEC_TRIGRAMME["Trigramme"].tolist():
        x['type_d_indicatif'] = "TR"
        if trouver_pattern(call_sign_value, INDICATIFS_TRIGRAMME_A_VERIFIER["Indicatif"]):
            x['PLN_valide'] = False
            x['invalidite_TU'].extend(["IVDIC5", "EXO5"])
        elif call_sign_value[:3] == "AFR":
            x['compagnie_française'] = True
            if re.search(r'\d[A-Z]$', call_sign_value):
                x = TU_611_element(x)
            else:
                x = TU_65_element(x)
        else:
            x = TU_612_element(x)
    else:
        x = TU_64_element(x)
    return x

In [2593]:
def TU_6(df_utile):
    def TU_6_element(x):
        def get_valid_value(element, columns):
            for col in columns:
                if col in element and not pd.isna(element[col]) and element[col] != "":
                    return element[col]
            return None
        call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
        if call_sign_value and trouver_pattern(call_sign_value, INDICATIF_A_CODE_AUTO["Indicatif"].tolist()):
            Indicatif = renvoie_pattern(call_sign_value, INDICATIF_A_CODE_AUTO["Indicatif"].tolist())
            x["code_d_exoneration"] = INDICATIF_A_CODE_AUTO[INDICATIF_A_CODE_AUTO["Indicatif"] == Indicatif]["Code exoneration"].iloc[0]
            x['code_exploitant'] = INDICATIF_A_CODE_AUTO[INDICATIF_A_CODE_AUTO["Indicatif"] == Indicatif]["Code exploitant"].iloc[0]
            if x["code_d_exoneration"] in ["X", "M"]:
                x['type_d_avion_militaire'] = True
        else:
            if trouver_pattern(call_sign_value, INDICATIFS_A_STRUCTURE_TRIGRAMME_CORRECTE["Indicatif"].tolist()):
                x = TU_61_element(x)
            elif re.match(r'^[A-Z]{2}\d{3}[A-Z]{2}$', call_sign_value) and call_sign_value[:2] in COMPAGNIES_BIGRAMMES_ET_SUFFIXES["Bigramme"].tolist():
                x = TU_62_element(x)
            else:
                x = TU_63_element(x)
        return x
    df_utile = df_utile.apply(TU_6_element, axis=1)
    return df_utile

## Algo 7

In [2594]:
# Chaîne d'entrée
def trouver_case18(prefix, input_string):
    index_find = input_string.find(prefix)
    if index_find == -1:
        return "NULL"
    else:
        start_index = index_find + len(prefix)
        end_index = input_string.find(' ', start_index)
        opr_code = input_string[start_index:end_index]
        return opr_code

In [2595]:
# def TU_7(df_utile):
#     def TU_7_element(x):
#         if x["PLN_activeprevu"] == "0" and x['vol_a_transmettre'] == True:
#             x['PLN_valide'] = False
#         if x["callSign_prevu"] == "APL" or x["callSign_prevu"] == "FPL":
#             if x['vol_a_transmettre'] == True:
#                 if x["code_d_exoneration"] == "Y" or ["code_d_exoneration"] == "Z":
#                     if x['typeVol_prevu'] == "M":
#                         x['PLN_valide'] = False
#                         x['invalidite_TU'].append("EXO10")
#                         x['invalidite_TU'].append("OPR10")
#                     else:
#                         if trouver_case18("RMK/", x["case18"]) == "TRAINING" and x['vol_interieur'] == True:
#                             x["code_d_exoneration"] = "T"
#                         elif x['typeVol_prevu'] == "X" and trouver_case18("RMK/", x["case18"]) != "EVASAN":
#                             x['PLN_valide'] = trouver_case18
#                             x['invalidite_TU'].append("EXO12")
#             else:
#                 if trouver_case18("RMK/", x["case18"]) == "TRAINING" and x['vol_interieur'] == True and (x["code_d_exoneration"] == "Y" or ["code_d_exoneration"] == "Z"):
#                     x["code_d_exoneration"] = "T"

#     df_utile = df_utile.apply(TU_7_element, axis=1)
#     return df_utile

In [2596]:
def TU_7(df_utile):
    def TU_7_element(x):
        # Fonction pour trouver une valeur valide dans les colonnes multiples
        def get_valid_value(element, columns):
            for col in columns:
                if col in element and not(element[[col]].isna().iloc[0]):
                    return element[col]
            return None

        PLN_active_value = get_valid_value(x, ["plnActive_realise", "plnActive_final", "plnActive_prevu"])
        call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
        type_vol_value = get_valid_value(x, ["typeVol_realise", "typeVol_final", "typeVol_prevu"])

        if PLN_active_value == "0" and x.get('vol_a_transmettre', False):
            x['PLN_valide'] = False

        if call_sign_value in ["APL", "FPL"]:
            if x.get('vol_a_transmettre', False):
                if code_exoneration_value in ["Y", "Z"]:
                    if type_vol_value == "M":
                        x['PLN_valide'] = False
                        x['invalidite_TU'].extend(["EXO10", "OPR10"])
                    else:
                        if trouver_case18("RMK/", x["case18"]) == "TRAINING" and x.get('vol_interieur', False):
                            x["code_d_exoneration"] = "T"
                        elif type_vol_value == "X" and trouver_case18("RMK/", x["case18"]) != "EVASAN":
                            x['PLN_valide'] = False
                            x['invalidite_TU'].append("EXO12")
            else:
                if trouver_case18("RMK/", x["case18"]) == "TRAINING" and x.get('vol_interieur', False) and code_exoneration_value in ["Y", "Z"]:
                    x["code_d_exoneration"] = "T"
        return x
    df_utile = df_utile.apply(TU_7_element, axis=1)
    return df_utile

## Traitement unitaire complet

In [2597]:
def traitement_unitaire(df_utile):
    df_utile = TU_init(df_utile)
    df_utile = TU_1(df_utile)
    df_utile = TU_2(df_utile)

    def check_depfinal_prefix(row):
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in row and not pd.isna(row[col]) and row[col] != "":
                if row[col][:2] == "LF":
                    return True
        return False

    def apply_transformations(row):
        if check_depfinal_prefix(row):
            return TU_4(pd.DataFrame([row])).iloc[0]
        else:
            return TU_3_bis(pd.DataFrame([row])).iloc[0]

    # Appliquer les transformations par ligne
    df_utile = df_utile.apply(apply_transformations, axis=1)

    df_utile = TU_5(df_utile)
    df_utile = TU_6(df_utile)
    df_utile = TU_7(df_utile)
    return df_utile

# Transformation des données RDVC

In [2598]:
jour_1_utile_inutile = traitement_utile_inutile(jour_1)
jour_2_utile_inutile = traitement_utile_inutile(jour_2)
jour_3_utile_inutile = traitement_utile_inutile(jour_3)

C:\Users\latti\AppData\Local\Temp\ipykernel_52804\2026010207.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['utile_inutile'] = df.apply(lambda x: utile_inutile(x), axis=1)
C:\Users\latti\AppData\Local\Temp\ipykernel_52804\2026010207.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['utile_inutile'] = df.apply(lambda x: utile_inutile(x), axis=1)


In [2599]:
jour_1_utile = jour_1_utile_inutile[jour_1_utile_inutile["utile_inutile"] == "UTI"].copy()
jour_2_utile = jour_2_utile_inutile[jour_2_utile_inutile["utile_inutile"] == "UTI"].copy()
jour_3_utile = jour_3_utile_inutile[jour_3_utile_inutile["utile_inutile"] == "UTI"].copy()

In [2600]:
jour_1_utile_traitee = traitement_unitaire(jour_1_utile)
jour_2_utile_traitee = traitement_unitaire(jour_2_utile)
jour_3_utile_traitee = traitement_unitaire(jour_3_utile)

Fonction
ajouter pour chaque df une colonne doublon à False
Verifier que si un call sign et une heure de reference à 19mn pret (à parametrer) indiquer doublon sur l'un des vols seulement

In [2601]:
seuil_doublon = 19

In [2602]:
import pandas as pd

# def ajouter_colonne_doublon(df, tolerance_minutes=19):

#     df['doublon'] = False
    

#     df = df.sort_values(by=['callSign_prevu', 'heure_de_reference']).reset_index(drop=True)

#     i = 1
#     while i < len(df):
#         if df.at[i, 'callSign_prevu'] == df.at[i - 1, 'callSign_prevu']:
#             delta = abs((df.at[i, 'heure_de_reference'] - df.at[i - 1, 'heure_de_reference']))
#             print()
#             print('delta = ', delta)
#             if delta <= tolerance_minutes:
#                 df = df.drop(i).reset_index(drop=True)
#                 continue  # Réévaluer le même index i après la suppression et la réinitialisation de l'index
#         i += 1

#     return df
def ajouter_colonne_doublon(df, tolerance_minutes=19):

    df['doublon'] = False
    

    df = df.sort_values(by=['callSign_prevu', 'heure_de_reference']).reset_index(drop=True)

    i = 1
    while i < len(df):
        if (df.at[i, 'callSign_prevu'] == df.at[i - 1, 'callSign_prevu']) and \
           ((df.at[i, 'dep_prevu'] == df.at[i - 1, 'dep_prevu']) or \
           (df.at[i, 'arr_prevu'] == df.at[i - 1, 'arr_prevu'])):
            delta = abs((df.at[i, 'heure_de_reference'] - df.at[i - 1, 'heure_de_reference']))
            print()
            print('delta = ', delta)
            if delta <= tolerance_minutes:
                df = df.drop(i).reset_index(drop=True)
                continue  # Réévaluer le même index i après la suppression et la réinitialisation de l'index
        i += 1

    return df

def supprimer_doublons(df):
    return
    
    return df[df['doublon'] == False].drop(columns=['doublon'])

In [2603]:
jour_1_utile_traitee.head()

,callSign_prevu,dep_prevu,arr_prevu,numCautra_prevu,dateRelative_prevu,typeAvion_prevu,work_prevu,heuresDep_prevu,RFL_prevu,vitesse_prevu,EOBT_prevu,regleVol_prevu,typeVol_prevu,HeurePremiereBaliseActive_prevu,IFPL_prevu,plnActive_prevu,typePlnStan,plnAnnule_prevu,dateBlock_prevu,adresseModeS_prevu,numeroPLNMprevu,flightIDprevu,baliseprevu,HeurePremiereBalise_prevu,listeBalisesprevu,indicateurprevu,carteprevu,centreTraverséprevu,listeRangPremierprevu,callSign_final,dep_final,arr_final,numCautra_final,dateRelative_final,typeAvion_final,work_final,heuresDep_final,RFL_final,vitesse_final,EOBT_final,regleVol_final,typeVol_final,HeurePremiereBaliseActive_final,IFPL_final,plnActive_final,plnAnnule_final,dateBlock_final,adresseModeS_final,numeroPLNMfinal,flightIDfinal,balisefinal,HeurePremiereBalise_final,listeBalisesfinal,indicateurfinal,cartefinal,centreTraverséfinal,listeRangPremierfinal,callSign_realise,dep_realise,arr_realise,numCautra_realise,dateRelative_realise,typeAvion_realise,work_realise,heuresDep_realise,RFL_realise,vitesse_realise,EOBT_realise,regleVol_realise,typeVol_realise,HeurePremiereBaliseActive_realise,IFPL_realise,plnActive_realise,plnAnnule_realise,dateBlock_realise,adresseModeS_realise,numeroPLNMrealise,flightIDrealise,baliserealise,HeurePremiereBalise_realise,listeBalisesrealise,indicateurrealise,carterealise,centreTraversérealise,listeRangPremierrealise,rangTransactiontransaction,heure,minute,accuseTrttransaction,ccrArrival,finaltransaction,isPrevu,isRealise,isFinal,work1prevu,work1final,work1realise,case7,case8,case9,case10,case13,case15,case16,case18,typePln,heure_de_reference,date_de_reference,utile_inutile,aeronef_de_moins_de_deux_tonnes,vol_a_transmettre,vol_approche,vol_interieur,vol_frontalier,vol_VFR,code_d_exoneration,code_exploitant,compagnie_française,type_d_avion_militaire,PLN_à_verifier_TU,PLN_valide,RAZ_des_invalidites_de_type_TU,invalidite_TU,type_d_indicatif
0,QUID11,EGUN,EGUN,2194,-1,K35R,0000,-800,310,400,-800,I,S,0,AA55547595,0,FIE,0,220124,NaN,578,????????,JOCUZ,873,310,0,UB,"[PARI, REIM, EGGG]",1,QUID11,EGUN,EGUN,2194,-1,K35R,0000,-800,310,400,-800,I,S,0,AA55547595,0,0,220124,NaN,578,????????,JOCUZ,873,310,0,UB,"[PARI, REIM, EGGG]",1,QUID11,EGUN,EGUN,-1,-1,K35R,0000,-800,280,400,-800,I,S,836,AA55547595,1,0,220124,NaN,-1,????????,PTGEO,-604,289,0,UB,[REIM],1,1,17,04,17:04,PARI,QUID11,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-604,2024-01-22,UTI,False,False,False,False,False,False,M,201,False,True,False,True,False,[],
1,THY189,MMUN,LTFM,2808,-1,B789,0000,-540,380,483,-540,I,S,0,AA55541004,0,FPL,0,220124,NaN,-1,????????,LULOX,-86,390,0,S9,"[EGGG, BRST, REIM, ZURI]",1,THY189,MMUN,LTFM,2808,-1,B789,5203,-540,380,483,-540,I,S,0,AA55541004,1,0,220124,NaN,-1,????????,LULOX,-78,390,0,S9,"[EGGG, BRST, REIM, ZURI]",1,THY189,MMUN,LTFM,2808,-1,B789,5203,-540,410,483,-540,I,S,1375,AA55541004,1,0,220124,4BB189,-1,THY189,LULOX,-79,390,0,S9,"[EGGG, BRST, REIM, ZURI]",1,4,19,50,19:50,EGGG,THY189,True,True,True,TCLLI,TCLLI,TCLLI,THY189,IS,B789/H,SADE1E2FGHIJ1J4J5M1P1P2RWXYZ/LB1D1,MMUN1500,N0483F380 NUDAL UV106 PODON/N0485F370 UG765 MA...,NaN,NaN,NaN,-65,2024-01-22,UTI,False,False,False,False,False,False,Z,Z,False,False,False,True,False,[],TR
2,EZY62RK,HESH,EGKK,2532,-1,A20N,0000,-445,380,448,-445,I,S,0,AA55537687,0,FPL,0,220124,NaN,-1,????????,DITON,-167,380,0,HH,"[ZURI, REIM, EGGG]",1,EZY62RK,HESH,EGKK,2532,-1,A20N,2740,-445,380,448,-445,I,S,0,AA55537687,0,0,220124,NaN,-1,????????,DITON,-167,380,0,HH,"[ZURI, REIM, EGGG]",1,EZY62RK,HESH,EGKK,-1,-1,A20N,0000,-445,350,448,-445,I,S,1266,AA55537687,1,0,220124,NaN,-1,????????,PTGEO,-174,380,0,HH,[REIM],1,1,18,08,18:08,ZURI,EZY62RKLGUZLSI,True,True,True,GUZLS,GUZLS,GUZLS,EZY62RK,IS,A20N/M,SDE2E3FGHIJ1RWXYZ/LB1G1,HESH1635,N0448F380 KUPTI1L KUPTI L677 CVO L617 TANSA UL...,EGKK0530 EGGW,CODE/407FFF DAT/V DOF/240122 EET/LGGG0129 LAAA...,NaN,-174,2024-01-22,UTI,False,False,False,False,False,False,Z,Z,False,False,False,True,False,[],TR
3,BAW33A,LG

In [2604]:
jour_1_utile_traitee_avec_doublons_false = ajouter_colonne_doublon(jour_1_utile_traitee)
jour_2_utile_traitee_avec_doublons_false = ajouter_colonne_doublon(jour_2_utile_traitee)
jour_3_utile_traitee_avec_doublons_false = ajouter_colonne_doublon(jour_3_utile_traitee)

jour_1_utile_traitee_sans_doublons = supprimer_doublons(jour_1_utile_traitee_avec_doublons_false)
jour_2_utile_traitee_sans_doublons = supprimer_doublons(jour_2_utile_traitee_avec_doublons_false)
jour_3_utile_traitee_sans_doublons = supprimer_doublons(jour_3_utile_traitee_avec_doublons_false)

C:\Users\latti\AppData\Local\Temp\ipykernel_52804\806245473.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['doublon'] = False



delta =  10

delta =  1443

delta =  24

delta =  22

delta =  30

delta =  71

delta =  1453

delta =  6

delta =  0

delta =  9

delta =  294

delta =  18

delta =  18

delta =  20

delta =  1431

delta =  0

delta =  1360

delta =  1440

delta =  1439

delta =  1411

delta =  27

delta =  1421

delta =  31

delta =  61

delta =  59

delta =  40

delta =  10

delta =  11

delta =  0

delta =  17

delta =  1

delta =  3

delta =  52

delta =  25

delta =  35

delta =  20

delta =  22

delta =  49

delta =  26

delta =  29

delta =  76

delta =  50

delta =  15

delta =  23

delta =  59

delta =  14

delta =  15

delta =  77

delta =  13

delta =  84

delta =  32

delta =  391

delta =  399

delta =  135

delta =  75

delta =  9

delta =  36

delta =  0

delta =  9

delta =  56

delta =  553

delta =  35

delta =  29

delta =  7

delta =  47

delta =  59

delta =  120

delta =  75

delta =  20

delta =  62

delta =  5

delta =  19

delta =  45

delta =  174

delta =  37

delta =  28



C:\Users\latti\AppData\Local\Temp\ipykernel_52804\806245473.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['doublon'] = False



delta =  0.0

delta =  30.0

delta =  0.0

delta =  1439.0

delta =  0.0

delta =  29.0

delta =  1400.0

delta =  1427.0

delta =  1470.0

delta =  1397.0

delta =  1440.0

delta =  1438.0

delta =  0.0

delta =  22.0

delta =  308.0

delta =  45.0

delta =  95.0

delta =  40.0

delta =  58.0

delta =  43.0

delta =  1445.0

delta =  1444.0

delta =  1432.0

delta =  59.0

delta =  1445.0

delta =  10.0

delta =  3.0

delta =  25.0

delta =  15.0

delta =  72.0

delta =  89.0

delta =  30.0

delta =  81.0

delta =  50.0

delta =  88.0

delta =  59.0

delta =  77.0

delta =  13.0

delta =  1454.0

delta =  1457.0

delta =  1462.0

delta =  1434.0

delta =  46.0

delta =  137.0

delta =  134.0

delta =  110.0

delta =  35.0

delta =  25.0

delta =  3.0

delta =  1478.0

delta =  135.0

delta =  5.0

delta =  288.0

delta =  28.0

delta =  90.0

delta =  42.0

delta =  40.0

delta =  21.0

delta =  130.0

delta =  37.0

delta =  20.0

delta =  0.0

delta =  20.0

delta =  25.0

delta = 

C:\Users\latti\AppData\Local\Temp\ipykernel_52804\806245473.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['doublon'] = False



delta =  46

delta =  5

delta =  85

delta =  6

delta =  1444

delta =  44

delta =  72

delta =  31

delta =  44

delta =  26

delta =  39

delta =  50

delta =  55

delta =  10

delta =  0

delta =  0

delta =  0

delta =  25

delta =  6

delta =  1

delta =  4

delta =  0

delta =  31

delta =  0

delta =  12

delta =  77

delta =  35

delta =  107

delta =  86

delta =  35

delta =  81

delta =  4

delta =  1422

delta =  51

delta =  2036

delta =  16

delta =  45

delta =  31

delta =  16

delta =  470

delta =  58

delta =  9

delta =  144

delta =  26

delta =  63

delta =  331

delta =  105

delta =  15

delta =  26

delta =  323

delta =  15

delta =  1338

delta =  6

delta =  104

delta =  0

delta =  2

delta =  2

delta =  1

delta =  1

delta =  19

delta =  1308

delta =  49

delta =  32

delta =  89

delta =  45

delta =  2

delta =  16

delta =  22

delta =  0

delta =  0

delta =  69

delta =  86

delta =  10

delta =  18

delta =  33

delta =  20

delta =  1416



In [2605]:
jour_2_utile_traitee.head()

,callSign_prevu,dep_prevu,arr_prevu,numCautra_prevu,dateRelative_prevu,typeAvion_prevu,work_prevu,heuresDep_prevu,RFL_prevu,vitesse_prevu,EOBT_prevu,regleVol_prevu,typeVol_prevu,HeurePremiereBaliseActive_prevu,IFPL_prevu,plnActive_prevu,typePlnStan,plnAnnule_prevu,dateBlock_prevu,adresseModeS_prevu,numeroPLNMprevu,flightIDprevu,baliseprevu,HeurePremiereBalise_prevu,listeBalisesprevu,indicateurprevu,carteprevu,centreTraverséprevu,listeRangPremierprevu,callSign_final,dep_final,arr_final,numCautra_final,dateRelative_final,typeAvion_final,work_final,heuresDep_final,RFL_final,vitesse_final,EOBT_final,regleVol_final,typeVol_final,HeurePremiereBaliseActive_final,IFPL_final,plnActive_final,plnAnnule_final,dateBlock_final,adresseModeS_final,HeurePremiereBalise_final,callSign_realise,dep_realise,arr_realise,numCautra_realise,dateRelative_realise,typeAvion_realise,work_realise,heuresDep_realise,RFL_realise,vitesse_realise,EOBT_realise,regleVol_realise,typeVol_realise,HeurePremiereBaliseActive_realise,IFPL_realise,plnActive_realise,plnAnnule_realise,dateBlock_realise,adresseModeS_realise,numeroPLNMrealise,flightIDrealise,baliserealise,HeurePremiereBalise_realise,listeBalisesrealise,indicateurrealise,carterealise,centreTraversérealise,listeRangPremierrealise,rangTransactiontransaction,heure,minute,accuseTrttransaction,ccrArrival,finaltransaction,isPrevu,isRealise,isFinal,work1prevu,work1final,numeroPLNMfinal,flightIDfinal,balisefinal,listeBalisesfinal,indicateurfinal,cartefinal,centreTraverséfinal,listeRangPremierfinal,work1realise,case7,case8,case9,case10,case13,case15,case16,case18,typePln,heure_de_reference,date_de_reference,utile_inutile,aeronef_de_moins_de_deux_tonnes,vol_a_transmettre,vol_approche,vol_interieur,vol_frontalier,vol_VFR,code_d_exoneration,code_exploitant,compagnie_française,type_d_avion_militaire,PLN_à_verifier_TU,PLN_valide,RAZ_des_invalidites_de_type_TU,invalidite_TU,type_d_indicatif,doublon
0,FAF7160,LFSI,LFSI,4508,-1,RFAL,0000,-1290,190,450,-1290,I,M,0,AA55474277,0,FPL,0,230124,NaN,2943,????????,EPL,-1275,190,0,E,"[REIM, FFOR]",1,FAF7160,LFSI,LFSI,4508,-1,RFAL,0000,-1290,190,450,-1290,I,M,0,AA55474277,0,0,230124,NaN,-1275,FAF7160,LFSI,LFSI,-1,-1,RFAL,6706,-1280,180,450,-1280,I,M,164,AA55474277,1,0,230124,NaN,-1,????????,PTGEO,-1276,200,0,E,[REIM],1,1,06,15,06:15,REIM,FAF7160C,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1276.0,2024-01-23,UTI,False,True,False,True,False,False,M,003,False,True,False,True,False,[],,False
1,DLH09M,EDDF,LFRS,8093,-1,CRJ9,0000,-540,340,443,-540,I,S,0,AA55552570,0,FII,0,230124,3C4DD0,6850,????????,DITAL,-473,330,0,XR,"[BRST, REIM, PARI, EDDY]",1,DLH09M,EDDF,LFRS,8093,-1,CRJ9,1000,-540,340,443,-540,I,S,0,AA55552570,1,0,230124,3C4DD0,-473,DLH09M,EDDF,LFRS,8093,-1,CRJ9,1000,-540,330,443,-540,I,S,967,AA55552570,1,0,230124,3C4DD0,6850,DLH09M,DITAL,-473,330,1,XR,"[BRST, REIM, PARI, EDDY]",1,1,16,08,16:08,BRST,DLH09M,True,True,True,DACNP,DACNP,6850,????????,DITAL,330,0,KR,"[BRST, REIM, PARI, EDDY]",1,DACNP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-473.0,2024-01-23,UTI,False,False,False,False,False,False,Z,Z,False,False,False,True,False,[],TR,False
2,AIC104,KIAD,VIDP,8650,-1,B788,0000,-525,350,488,-525,I,S,0,AA55565142,0,FPL,0,230124,NaN,-1,????????,SANDY,-150,370,0,HN,"[EGGG, REIM, ZURI]",1,AIC104,KIAD,VIDP,8650,-1,B788,1153,-525,350,488,-525,I,S,0,AA55565142,0,0,230124,NaN,-165,AIC104,KIAD,VIDP,-1,-1,B788,0000,-525,390,488,-525,I,S,1266,AA55565142,0,0,230124,NaN,-1,????????,PTGEO,-174,390,0,HN,[REIM],1,2,18,09,18:09,EGGG,AIC104,True,True,True,VTANY,VTANY,-1,????????,SANDY,370,0,HN,"[EGGG, REIM, ZURI]",1,VTANY,AIC104,IS,B788/H,SDE1E2E3FGHIJ1J3J4J5M1P2RWXYZ/LB1D1,KIAD1515,N0488F350 JERES2 JERES DCT ALB DCT EBONY DCT A...,NaN,NaN,NaN,-174.0,2024-01-23,UTI,False,False,False,False,False,False,Z,Z,False,False,False,True,False,[NACT],TR,False
3,EZY95GR,HEGN,EGKK,8458,-1,A21N,0000,-495,340,455,-495,I,S,0,AA55561946,0,FPL,0,230124,NaN,-1,????????,DITON,-227,3

In [2606]:
def get_valid_value(element, columns):
        for col in columns:
            if col in element and not(element[[col]].isna().iloc[0]):
                return element[col]
        return ""

def format_flight_message(row):
    sequence_number = f"{row.name + 1:04}"
    intro_correction_code = "F"
    valeur = int(get_valid_value(row, ['heure_de_reference']))
    heure = (valeur%1440)//60
    min = (valeur%1440)%60
    time_of_departure = f"{(100*heure + min):04}"
    aerodrome_of_departure = get_valid_value(row, ['dep_realise', 'dep_final', 'de_prevu']).ljust(4)
    aerodrome_of_arrival = get_valid_value(row, ['arr_realise', 'arr_final', 'arr_prevu']).ljust(4)
    flight_identification = get_valid_value(row, ['callSign_realise', 'callSign_final', 'callSign_prevu']).ljust(9)
    main_exemption_code = row["code_d_exoneration"].ljust(1)
    aircraft_type = get_valid_value(row, ['typeAvion_realise', 'typeAvion_final', 'typeAvion_prevu']).ljust(7)
    operator = str(row["code_exploitant"]).ljust(3)
    aircraft_registration = str(get_valid_value(row, ['work1realise', 'work1final', 'work1prevu'])).ljust(9)
    comment = ""
    est_off_block_date = get_valid_value(row, ['date_de_reference']).ljust(6)
    ifplid = get_valid_value(row, ['IFPL_realise', 'IFPL_final', 'IFPL_prevu']).ljust(9)
    initial_aerodrome_destination = ""
    charging_zone_overflown = ""
    entry_point_coordinates = ""
    exit_point_coordinates = ""
    supplementary_exemption_codes = ""
    source_icao_address = ""
    icao_address = ""
    additional_comment = ""

    return {
        "Sequence number": sequence_number,
        "Code": intro_correction_code,
        "Time of departure/entry": time_of_departure,
        "Departure aerodrome": aerodrome_of_departure,
        "Arrival aerodrome": aerodrome_of_arrival,
        "Flight identification": flight_identification,
        "Main Exemption code": main_exemption_code,
        "Type of aircraft": aircraft_type,
        "Operator": operator,
        "Aircraft Registration": aircraft_registration,
        "Comment1": comment,
        "Flight date": est_off_block_date,
        "IFPLID": ifplid,
        "Planned_aerodrome": initial_aerodrome_destination,
        "Charging_zone_overflow": charging_zone_overflown,
        "Entry_point": entry_point_coordinates,
        "Exit_point": exit_point_coordinates,
        "Sup_exemption_code": supplementary_exemption_codes,
        "Source of the Aircraft Address": source_icao_address,
        "24-bit Aircraft Address": icao_address,
        "Comment2": additional_comment
    }


In [2607]:
jour_123_brut = pd.concat([jour_1_utile_traitee_avec_doublons_false, jour_2_utile_traitee_avec_doublons_false, jour_3_utile_traitee_avec_doublons_false], ignore_index=True)
jour_123_brut['date_de_reference'] = jour_123_brut['date_de_reference'].dt.strftime('%y%m%d')
jour_123 = jour_123_brut[jour_123_brut["vol_a_transmettre"] == True]
jour_123 = jour_123.apply(format_flight_message, axis=1).tolist()
jour_123 = pd.DataFrame(jour_123)
jour_123['Time of departure/entry'] = pd.to_numeric(jour_123['Time of departure/entry'], errors='coerce')

# Gestion consécutifs

In [2608]:
jour_123.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Sequence number                 6040 non-null   object
 1   Code                            6040 non-null   object
 2   Time of departure/entry         6040 non-null   int64 
 3   Departure aerodrome             6040 non-null   object
 4   Arrival aerodrome               6040 non-null   object
 5   Flight identification           6040 non-null   object
 6   Main Exemption code             6040 non-null   object
 7   Type of aircraft                6040 non-null   object
 8   Operator                        6040 non-null   object
 9   Aircraft Registration           6040 non-null   object
 10  Comment1                        6040 non-null   object
 11  Flight date                     6040 non-null   object
 12  IFPLID                          6040 non-null   

In [2609]:
jour_123.head()

,Sequence number,Code,Time of departure/entry,Departure aerodrome,Arrival aerodrome,Flight identification,Main Exemption code,Type of aircraft,Operator,Aircraft Registration,Comment1,Flight date,IFPLID,Planned_aerodrome,Charging_zone_overflow,Entry_point,Exit_point,Sup_exemption_code,Source of the Aircraft Address,24-bit Aircraft Address,Comment2
0,0001,F,1311,LFML,LFML,125F,Z,AS32,Z,,,240123,,,,,,,,,
1,0002,F,1653,LFML,LFML,160O,Z,AS32,Z,,,240123,,,,,,,,,
2,0004,F,835,LFGJ,LFOV,2NYAW,Z,FA50,Z,2NYAW,,240123,AA55564598,,,,,,,,
3,0005,F,1300,LFOV,LFRG,2NYAW,Z,FA50,Z,2NYAW,,240123,AA55564706,,,,,,,,
4,0006,F,1700,LFRG,LFOV,2NYAW,Z,FA50,Z,2NYAW,,240123,AA55564794,,,,,,,,


# Donnees ABACUS

In [2610]:
liste_noms_colonnes = ['Origin of message','Flight date','Sequence number','Time of departure/entry','Departure aerodrome','Arrival aerodrome','Type of aircraft','Flight identification','Aircraft Registration','User ICAO Code','User Number','VAT Number','User Nationality','Correction Code','IFPLID','24-bit Aircraft Address','Source of the Aircraft Address','Main Exemption Code','1st Supp. Ex. Code','2nd Supp. Ex. Code','3rd Supp. Ex. Code','Flight Message ID','Claim Number','Pro forma number','Pro forma line number','Bill or Credit Note Reference','MTOW','Weight Coefficient','Distance Coefficient','Service Units','National Route Charge in BZ','Admin. Route Charge in BZ','National Route Charge for SP','Admin. Route Charge for SP','Exemption Indicator','Exemption Code','Flight category','VAT Doc. Ref.','Original VAT Doc. Ref','VAT Rate','VAT Code','VAT on National Route Charge for SP','VAT on Admin. Route Charge for SP']
# ABACUS_Mars = pd.read_csv('../data/ABACUS/LF20LF10%S%ABACUS_FLSPBZ03_2403.TXT', sep=';', usecols = liste_noms_colonnes)

In [2611]:
# ABACUS_Mars_toutes_origines = ABACUS_Mars[ABACUS_Mars["Flight date"] == 240316]
# ABACUS_Mars = ABACUS_Mars[(ABACUS_Mars["Flight date"] == 240316) & (ABACUS_Mars["Origin of message"] == "LF")]

In [2612]:
def get_valid_value(element, columns):
        for col in columns:
            if col in element and not(element[[col]].isna().iloc[0]):
                return element[col]
        return ""

def format_flight_message2(row):
    sequence_number = f"{row.name + 1:04}"
    intro_correction_code = "F"
    time_of_departure = f"{(int(get_valid_value(row, ['Time of departure/entry']))):04}"
    aerodrome_of_departure = get_valid_value(row, ['Departure aerodrome']).ljust(4)
    aerodrome_of_arrival = get_valid_value(row, ['Arrival aerodrome']).ljust(4)
    flight_identification = get_valid_value(row, ['Flight identification']).ljust(9)
    main_exemption_code = row['Main Exemption Code'].ljust(1)
    aircraft_type = get_valid_value(row, ['Type of aircraft']).ljust(7)
    operator = row['Main Exemption Code'].ljust(3)
    aircraft_registration = get_valid_value(row, ['Aircraft Registration']).ljust(9)
    comment = ""
    est_off_block_date = str(get_valid_value(row, ['Flight date'])).ljust(6)
    ifplid = get_valid_value(row, ['IFPLID']).ljust(9)
    initial_aerodrome_destination = ""
    charging_zone_overflown = ""
    entry_point_coordinates = ""
    exit_point_coordinates = ""
    supplementary_exemption_codes = ""
    source_icao_address = row['Source of the Aircraft Address']
    icao_address = row['24-bit Aircraft Address']
    additional_comment = ""

    return {
        "Sequence number": sequence_number,
        "Code": intro_correction_code,
        "Time of departure/entry": time_of_departure,
        "Departure aerodrome": aerodrome_of_departure,
        "Arrival aerodrome": aerodrome_of_arrival,
        "Flight identification": flight_identification,
        "Main Exemption code": main_exemption_code,
        "Type of aircraft": aircraft_type,
        "Operator": operator,
        "Aircraft Registration": aircraft_registration,
        "Comment1": comment,
        "Flight date": est_off_block_date,
        "IFPLID": ifplid,
        "Planned_aerodrome": initial_aerodrome_destination,
        "Charging_zone_overflow": charging_zone_overflown,
        "Entry_point": entry_point_coordinates,
        "Exit_point": exit_point_coordinates,
        "Sup_exemption_code": supplementary_exemption_codes,
        "Source of the Aircraft Address": source_icao_address,
        "24-bit Aircraft Address": icao_address,
        "Comment2": additional_comment
    }

In [2613]:
# ABACUS_Mars = ABACUS_Mars.apply(format_flight_message2, axis=1).tolist()
# # Créer un nouveau DataFrame à partir de la liste de dictionnaires
# ABACUS_Mars = pd.DataFrame(ABACUS_Mars)

# ABACUS_Mars['Time of departure/entry'] = pd.to_numeric(ABACUS_Mars['Time of departure/entry'], errors='coerce')

# Fonction de comparaison

In [2614]:
"""## Importation données
Données DTI et CESNAC
"""

widths= [4,1,4,4,4,9,1,7,3,9,21,6,10,4,2,15,15,3,1,6,76]
noms_colonnes = ["Sequence number", "Code", "Time of departure/entry", "Departure aerodrome", "Arrival aerodrome",
                 "Flight identification", "Main Exemption code", "Type of aircraft", "Operator", "Aircraft Registration",
                 "Comment1", "Flight date", "IFPLID", "Planned_aerodrome", "Charging_zone_overflow", "Entry_point", "Exit_point",
                 "Sup_exemption_code", "Source of the Aircraft Address", "24-bit Aircraft Address", "Comment2"]
noms_colonnes_doubles = [f"{x}_{i+1}" for x in noms_colonnes for i in range(2)]

"""# Algorithme de comparaison

## Fonction calcul heures de différences
"""

def difference_time(heure1, heure2, date1, date2, date_fichier1, date_fichier2):
    if heure1 <10:
      h1 = "000" + str(int(heure1))
    elif heure1 < 100:
      h1 = "00" + str(int(heure1))
    elif heure1 < 1000:
      h1 = "0" + str(int(heure1))
    else:
      h1 = str(int(heure1))
    if heure2 <10:
      h2 = "000" + str(int(heure2))
    elif heure2 < 100:
      h2 = "00" + str(int(heure2))
    elif heure2 < 1000:
      h2 = "0" + str(int(heure2))
    else:
      h2 = str(int(heure2))
    try:
      date_obj1 = datetime.strptime(str(int(date1)) + h1, '%y%m%d%H%M')
      date_obj2 = datetime.strptime(str(int(date2)) + h2, '%y%m%d%H%M')
    except ValueError:
        # Si les dates ne sont pas dans le bon format, supposer qu'elles sont les mêmes
        date1 = date_fichier1
        date2 = date_fichier2
        # Convertir les dates en objets datetime
        date_obj1 = datetime.strptime(str(int(date1)) + h1, '%y%m%d%H%M')
        date_obj2 = datetime.strptime(str(int(date2)) + h2, '%y%m%d%H%M')

    # Calculer la différence entre les deux horaires
    difference = date_obj1 - date_obj2

    #Avoir la différence en seconde
    difference_seconde = abs(difference.days * 3600*24 + difference.seconds)
    # Convertir la différence en heures et minutes
    difference_heures = difference_seconde // 3600
    difference_minutes = (difference_seconde % 3600) // 60

    # Retourner la différence sous forme d'entier hhmm
    return difference_heures * 100 + difference_minutes

"""## Définition vol similaire"""

def comparaison_vol(vol1, vol2, fichier_date1, fichier_date2) -> bool:
  arrival = (vol1["Arrival aerodrome"] == vol2["Arrival aerodrome"])
  departure = (vol1["Departure aerodrome"] == vol2["Departure aerodrome"])
  id = (vol1["Flight identification"] == vol2[ "Flight identification"])
  if (arrival and departure and id) :
    return(difference_time(vol1["Time of departure/entry"], vol2["Time of departure/entry"], vol1["Flight date"], vol2["Flight date"], fichier_date1, fichier_date2) < 45)
  else:
    return False

def comparaison_vol_arrival(vol1, vol2, fichier_date1, fichier_date2) -> bool:
  arrival = (vol1["Arrival aerodrome"] == vol2["Arrival aerodrome"])
  id = (vol1["Flight identification"] == vol2[ "Flight identification"])
  if (arrival and id) :
    return(difference_time(vol1["Time of departure/entry"], vol2["Time of departure/entry"], vol1["Flight date"], vol2["Flight date"], fichier_date1, fichier_date2) < 20)
  else:
    return False

def comparaison_vol_departure(vol1, vol2, fichier_date1, fichier_date2) -> bool:
  departure = (vol1["Departure aerodrome"] == vol2["Departure aerodrome"])
  id = (vol1["Flight identification"] == vol2[ "Flight identification"])
  if (departure and id) :
    return(difference_time(vol1["Time of departure/entry"], vol2["Time of departure/entry"], vol1["Flight date"], vol2["Flight date"], fichier_date1, fichier_date2) < 20)
  else:
    return False

"""## Retrouver vol(s) dans dataset"""

def retrouver_vol(vol, data,fichier_date1, fichier_date2):
    matching_rows = data[data.apply(lambda row: comparaison_vol(vol, row, fichier_date1, fichier_date2), axis=1)]
    index = matching_rows.index
    return ([matching_rows.iloc[0]], index[0]) if len(matching_rows) >0 else ([],0)

def retrouver_vol_arrival(vol, data,fichier_date1, fichier_date2):
    matching_rows = data[data.apply(lambda row: comparaison_vol_arrival(vol, row, fichier_date1, fichier_date2), axis=1)]
    index = matching_rows.index
    return ([matching_rows.iloc[0]], index[0]) if len(matching_rows) >0 else ([],0)

def retrouver_vol_departure(vol, data,fichier_date1, fichier_date2):
    matching_rows = data[data.apply(lambda row: comparaison_vol_departure(vol, row, fichier_date1, fichier_date2), axis=1)]
    index = matching_rows.index
    return ([matching_rows.iloc[0]], index[0]) if len(matching_rows) >0 else ([],0)

"""## Différences entre 2 vols"""

def differences_vols(vol1, vol2):
  diff = []
  noms = noms_colonnes.copy()
  noms.remove("Sequence number")
  for name in noms:
    if vol1[name] != vol2[name] and not(pd.isna(vol1[name]) and pd.isna(vol2[name])) :
      diff.append(name)
  if len(diff)>0:
    return True
  else:
    return False

"""## Comparaison entre deux datasets d'une journée de vol, format voulu par Eurocontrol"""

def comparaison_data(data1, data2, fichier_date1, fichier_date2):

  # Premières comparaison
  vols_only_data1 = pd.DataFrame(columns=noms_colonnes)
  vols_only_data2 = data2.copy()
  vols_identiques = pd.DataFrame(columns = noms_colonnes)
  index_identiques = 0
  vols_differents = pd.DataFrame(columns=noms_colonnes_doubles)
  for i in range(len(data1)):
    vols, index = retrouver_vol(data1.iloc[i], vols_only_data2, fichier_date1, fichier_date2)
    if len(vols)==0:
      vols_only_data1 = pd.concat([vols_only_data1, data1.iloc[i:i+1]], ignore_index=True)
    else:
      vols_only_data2 = vols_only_data2.drop(index)
      if differences_vols(data1.iloc[i], vols[0]):
        # Convertir les séries en DataFrame
        data1_df = data1.iloc[i].to_frame().T
        vols_df = vols[0].to_frame().T
        # Ajouter un suffixe aux colonnes de la deuxième ligne
        data1_df.columns = [colonne + '_1' for colonne in data1_df.columns]
        vols_df.columns = [colonne + '_2' for colonne in vols_df.columns]
        # Réinitialiser les index pour aligner les lignes
        data1_df.reset_index(drop=True, inplace=True)
        vols_df.reset_index(drop=True, inplace=True)
        # Créer une nouvelle ligne en alternant les valeurs
        newligne = pd.DataFrame()
        for colonne1, colonne2 in zip(data1_df.columns, vols_df.columns):
            newligne[colonne1] = data1_df[colonne1]
            newligne[colonne2] = vols_df[colonne2]
        # Réinitialiser l'index pour obtenir une seule ligne
        newligne.reset_index(drop=True, inplace=True)
        vols_differents = pd.concat([vols_differents, newligne], ignore_index=True)
      else:
        vols_identiques.loc[index_identiques] = data1.iloc[i]
        index_identiques+=1

  # Premières comparaison
  vols_only_only_data1 = pd.DataFrame(columns=noms_colonnes)
  vols_only_only_data2 = vols_only_data2.copy()
  vols_arr_identique = pd.DataFrame(columns=noms_colonnes_doubles)
  vols_dep_identique = pd.DataFrame(columns=noms_colonnes_doubles)
  for i in range(len(vols_only_data1)):
    vols, index = retrouver_vol_departure(vols_only_data1.iloc[i], vols_only_only_data2, fichier_date1, fichier_date2)
    if len(vols)==0:
      vols, index = retrouver_vol_arrival(vols_only_data1.iloc[i], vols_only_only_data2, fichier_date1, fichier_date2)
      if len(vols)==0:
        vols_only_only_data1 = pd.concat([vols_only_only_data1, vols_only_data1.iloc[i:i+1]], ignore_index=True)
      else:
        vols_only_only_data2 = vols_only_only_data2.drop(index)
        # Convertir les séries en DataFrame
        data1_df = vols_only_data1.iloc[i].to_frame().T
        vols_df = vols[0].to_frame().T
        # Ajouter un suffixe aux colonnes de la deuxième ligne
        data1_df.columns = [colonne + '_1' for colonne in data1_df.columns]
        vols_df.columns = [colonne + '_2' for colonne in vols_df.columns]
        # Réinitialiser les index pour aligner les lignes
        data1_df.reset_index(drop=True, inplace=True)
        vols_df.reset_index(drop=True, inplace=True)
        # Créer une nouvelle ligne en alternant les valeurs
        newligne = pd.DataFrame()
        for colonne1, colonne2 in zip(data1_df.columns, vols_df.columns):
            newligne[colonne1] = data1_df[colonne1]
            newligne[colonne2] = vols_df[colonne2]
        # Réinitialiser l'index pour obtenir une seule ligne
        newligne.reset_index(drop=True, inplace=True)
        vols_arr_identique = pd.concat([vols_arr_identique, newligne], ignore_index=True)
    else:
      vols_only_only_data2 = vols_only_only_data2.drop(index)
      # Convertir les séries en DataFrame
      data1_df = vols_only_data1.iloc[i].to_frame().T
      vols_df = vols[0].to_frame().T
      # Ajouter un suffixe aux colonnes de la deuxième ligne
      data1_df.columns = [colonne + '_1' for colonne in data1_df.columns]
      vols_df.columns = [colonne + '_2' for colonne in vols_df.columns]
      # Réinitialiser les index pour aligner les lignes
      data1_df.reset_index(drop=True, inplace=True)
      vols_df.reset_index(drop=True, inplace=True)
      # Créer une nouvelle ligne en alternant les valeurs
      newligne = pd.DataFrame()
      for colonne1, colonne2 in zip(data1_df.columns, vols_df.columns):
          newligne[colonne1] = data1_df[colonne1]
          newligne[colonne2] = vols_df[colonne2]
      # Réinitialiser l'index pour obtenir une seule ligne
      newligne.reset_index(drop=True, inplace=True)
      vols_dep_identique = pd.concat([vols_dep_identique, newligne], ignore_index=True)

  vols_only_only_data2_moins_2T = pd.DataFrame(columns=noms_colonnes)
  vols_only_only_data1_moins_2T = pd.DataFrame(columns=noms_colonnes)
  vols_only_only_data1_copy = vols_only_only_data1.copy()
  vols_only_only_data2_copy = vols_only_only_data2.copy()
  for i in range(len(vols_only_only_data1_copy)):
      if vols_only_only_data1_copy["Type of aircraft"].iloc[i].rstrip() in AERONEFS_DE_MOINS_DE_2_TONNES["Type avion"].to_list():
          vols_only_only_data1_moins_2T = pd.concat([vols_only_only_data1_moins_2T, vols_only_only_data1_copy.iloc[i:i+1]], ignore_index=True)
          vols_only_only_data1 = vols_only_only_data1.drop(vols_only_only_data1_copy.index[i])
  for i in range(len(vols_only_only_data2_copy)):
      if vols_only_only_data2_copy["Type of aircraft"].iloc[i].rstrip() in AERONEFS_DE_MOINS_DE_2_TONNES["Type avion"].to_list():
          vols_only_only_data2_moins_2T = pd.concat([vols_only_only_data2_moins_2T, vols_only_only_data2_copy.iloc[i:i+1]], ignore_index=True)
          vols_only_only_data2 = vols_only_only_data2.drop(vols_only_only_data2_copy.index[i])

  # Traitement final
  vols_differents.insert(0, 'compare', "vols_identiques")
  vols_identiques.insert(0, 'compare', "vols_quasi_identiques")
  vols_only_only_data1.insert(0, 'compare', "vols_only_data_1_plus_2T")
  vols_only_only_data2.insert(0, 'compare', "vols_only_data_2_plus_2T")
  vols_only_only_data1_moins_2T.insert(0, 'compare', "vols_only_data_1_moins_2T")
  vols_only_only_data2_moins_2T.insert(0, 'compare', "vols_only_data_2_moins_2T")
  vols_dep_identique.insert(0, 'compare', "vols_arr_diff")
  vols_arr_identique.insert(0, 'compare', "vols_dep_diff")

  colonnes_dupliquees = [vols_identiques.iloc[:, 0]] + [vols_identiques[colonne].rename(f"{colonne}_{i+1}") for colonne in vols_identiques.columns[1:] for i in range(2)]
  vols_identiques = pd.concat(colonnes_dupliquees, axis=1)

  vols_only_only_data1.reset_index(drop=True, inplace=True)
  colonnes_dupliquees = []
  colonnes_dupliquees.append(vols_only_only_data1.iloc[:, 0])
  for colonne in vols_only_only_data1.columns[1:]:
      colonnes_dupliquees.append(vols_only_only_data1[colonne].rename(f"{colonne}_1"))
      colonnes_dupliquees.append(pd.Series([np.nan] * len(vols_only_only_data1), name=f"{colonne}_2"))
  vols_only_only_data1 = pd.concat(colonnes_dupliquees, axis=1)

  vols_only_only_data2.reset_index(drop=True, inplace=True)
  colonnes_dupliquees = []
  colonnes_dupliquees.append(vols_only_only_data2.iloc[:, 0])
  for colonne in vols_only_only_data2.columns[1:]:
      colonnes_dupliquees.append(pd.Series([np.nan] * len(vols_only_only_data2), name=f"{colonne}_1"))
      colonnes_dupliquees.append(vols_only_only_data2[colonne].rename(f"{colonne}_2"))
  vols_only_only_data2 = pd.concat(colonnes_dupliquees, axis=1)

  vols_only_only_data1_moins_2T.reset_index(drop=True, inplace=True)
  colonnes_dupliquees = []
  colonnes_dupliquees.append(vols_only_only_data1_moins_2T.iloc[:, 0])
  for colonne in vols_only_only_data1_moins_2T.columns[1:]:
      colonnes_dupliquees.append(vols_only_only_data1_moins_2T[colonne].rename(f"{colonne}_1"))
      colonnes_dupliquees.append(pd.Series([np.nan] * len(vols_only_only_data1_moins_2T), name=f"{colonne}_2"))
  vols_only_only_data1_moins_2T = pd.concat(colonnes_dupliquees, axis=1)

  vols_only_only_data2_moins_2T.reset_index(drop=True, inplace=True)
  colonnes_dupliquees = []
  colonnes_dupliquees.append(vols_only_only_data2_moins_2T.iloc[:, 0])
  for colonne in vols_only_only_data2_moins_2T.columns[1:]:
      colonnes_dupliquees.append(pd.Series([np.nan] * len(vols_only_only_data2_moins_2T), name=f"{colonne}_1"))
      colonnes_dupliquees.append(vols_only_only_data2_moins_2T[colonne].rename(f"{colonne}_2"))
  vols_only_only_data2_moins_2T = pd.concat(colonnes_dupliquees, axis=1)

  result = pd.concat([vols_differents, vols_identiques, vols_only_only_data1, vols_only_only_data2, vols_only_only_data1_moins_2T, vols_only_only_data2_moins_2T, vols_dep_identique, vols_arr_identique])
  result = result.drop("Sequence number_1", axis=1)
  result = result.drop("Sequence number_2", axis=1)
  result = result.reset_index(drop=True)
  return(result)

# Result

In [2615]:
# jour_cible = jour_123[(jour_123["Flight date"] == "240316") | (jour_123["Flight date"] == "      ")]

In [2616]:
nom_abacus

'LF20LF10%S%ABACUS_FLSPBZ03_2401.TXT'

In [2617]:
target_date_str =dateAnalyse
target_date = datetime.strptime(target_date_str, "%d-%m-%Y")
target_date_str = target_date.strftime("%y%m%d")
ABACUS = pd.read_csv(f"../data/ABACUS/{nom_abacus}", sep=';', usecols = liste_noms_colonnes)
ABACUS_toutes_origines = ABACUS[ABACUS["Flight date"] == int(target_date_str)]

ABACUS = ABACUS[(ABACUS["Flight date"] == int(target_date_str)) & (ABACUS["Origin of message"] == "LF")]
ABACUS = ABACUS.apply(format_flight_message2, axis=1).tolist()
# Créer un nouveau DataFrame à partir de la liste de dictionnaires
ABACUS = pd.DataFrame(ABACUS)

ABACUS['Time of departure/entry'] = pd.to_numeric(ABACUS['Time of departure/entry'], errors='coerce')
# Filter jour_123 for the target date and blank dates
jour_cible = jour_123[(jour_123["Flight date"] == target_date_str) | (jour_123["Flight date"] == "      ")]
    

C:\Users\latti\AppData\Local\Temp\ipykernel_52804\1202911970.py:4: DtypeWarning: Columns (18,35) have mixed types. Specify dtype option on import or set low_memory=False.
  ABACUS = pd.read_csv(f"../data/ABACUS/{nom_abacus}", sep=';', usecols = liste_noms_colonnes)


In [2618]:
jour_123['Flight date'].value_counts()

Flight date
240124    2079
240125    2073
240123    1873
240122      15
Name: count, dtype: int64

In [2619]:
jour_cible.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2079 entries, 1863 to 5905
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Sequence number                 2079 non-null   object
 1   Code                            2079 non-null   object
 2   Time of departure/entry         2079 non-null   int64 
 3   Departure aerodrome             2079 non-null   object
 4   Arrival aerodrome               2079 non-null   object
 5   Flight identification           2079 non-null   object
 6   Main Exemption code             2079 non-null   object
 7   Type of aircraft                2079 non-null   object
 8   Operator                        2079 non-null   object
 9   Aircraft Registration           2079 non-null   object
 10  Comment1                        2079 non-null   object
 11  Flight date                     2079 non-null   object
 12  IFPLID                          2079 non-null   ob

In [2620]:
dataresult = comparaison_data(jour_cible, ABACUS, int(target_date_str), int(target_date_str))

In [2621]:
dataresult["compare"].value_counts()

compare
vols_identiques              1981
vols_only_data_1_plus_2T       77
vols_arr_diff                  15
vols_only_data_2_plus_2T       13
vols_only_data_1_moins_2T       5
vols_dep_diff                   1
Name: count, dtype: int64

In [2622]:
ABACUS['Flight date'].value_counts()

Flight date
240124    2010
Name: count, dtype: int64

In [2623]:
jour_cible.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2079 entries, 1863 to 5905
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Sequence number                 2079 non-null   object
 1   Code                            2079 non-null   object
 2   Time of departure/entry         2079 non-null   int64 
 3   Departure aerodrome             2079 non-null   object
 4   Arrival aerodrome               2079 non-null   object
 5   Flight identification           2079 non-null   object
 6   Main Exemption code             2079 non-null   object
 7   Type of aircraft                2079 non-null   object
 8   Operator                        2079 non-null   object
 9   Aircraft Registration           2079 non-null   object
 10  Comment1                        2079 non-null   object
 11  Flight date                     2079 non-null   object
 12  IFPLID                          2079 non-null   ob

In [2624]:
import pandas as pd

# Charger les données existantes
try:
    existing_data = pd.read_csv('vols_manquants.csv')
except FileNotFoundError:
    existing_data = pd.DataFrame()

# Filtrer les nouvelles données
new_data = dataresult[dataresult["compare"] == "vols_only_data_2_plus_2T"]
colonnes_a_afficher = ['compare', 'Flight date_2', 'Time of departure/entry_2', 'Departure aerodrome_2', 'Arrival aerodrome_2', 'Flight identification_2', 'Type of aircraft_2']


# Supprimer les lignes avec les mêmes valeurs de 'Flight date_2'
if not existing_data.empty:
    existing_data = existing_data[~existing_data['Flight date_2'].isin(new_data['Flight date_2'])]

# Ajouter les nouvelles données
updated_data = pd.concat([existing_data, new_data[colonnes_a_afficher]])

# Enregistrer dans le fichier CSV
updated_data.to_csv('vols_manquants_algo2.csv', index=False)

In [2625]:
dataresult[dataresult["compare"] == "vols_only_data_2_plus_2T"]

,compare,Code_1,Code_2,Time of departure/entry_1,Time of departure/entry_2,Departure aerodrome_1,Departure aerodrome_2,Arrival aerodrome_1,Arrival aerodrome_2,Flight identification_1,Flight identification_2,Main Exemption code_1,Main Exemption code_2,Type of aircraft_1,Type of aircraft_2,Operator_1,Operator_2,Aircraft Registration_1,Aircraft Registration_2,Comment1_1,Comment1_2,Flight date_1,Flight date_2,IFPLID_1,IFPLID_2,Planned_aerodrome_1,Planned_aerodrome_2,Charging_zone_overflow_1,Charging_zone_overflow_2,Entry_point_1,Entry_point_2,Exit_point_1,Exit_point_2,Sup_exemption_code_1,Sup_exemption_code_2,Source of the Aircraft Address_1,Source of the Aircraft Address_2,24-bit Aircraft Address_1,24-bit Aircraft Address_2,Comment2_1,Comment2_2
2058,vols_only_data_2_plus_2T,NaN,F,NaN,1039,NaN,LFPM,NaN,LFPM,NaN,FGBTP,NaN,Z,NaN,PA34,NaN,Z,NaN,FGBTP,NaN,,NaN,240124,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,1.0,NaN,39066F,NaN,
2059,vols_only_data_2_plus_2T,NaN,F,NaN,353,NaN,HUEN,NaN,EBBR,NaN,BEL4KE,NaN,Z,NaN,A333,NaN,Z,NaN,OOSFD,NaN,,NaN,240124,NaN,AA55571184,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,1.0,NaN,44CCC4,NaN,
2060,vols_only_data_2_plus_2T,NaN,F,NaN,1016,NaN,LFBU,NaN,LFBU,NaN,CTM1750,NaN,M,NaN,E121,NaN,M,NaN,FYSBG,NaN,,NaN,240124,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,
2061,vols_only_data_2_plus_2T,NaN,F,NaN,1412,NaN,LFRN,NaN,LFRN,NaN,FGN9330,NaN,M,NaN,EC35,NaN,M,NaN,FMJDA,NaN,,NaN,240124,NaN,AA55595312,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,1.0,NaN,3AAC12,NaN,
2062,vols_only_data_2_plus_2T,NaN,F,NaN,437,NaN,HKJK,NaN,EHBK,NaN,UAE9745,NaN,Z,NaN,B77L,NaN,Z,NaN,A6EFM,NaN,,NaN,240124,NaN,AA55575371,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,
2063,vols_only_data_2_plus_2T,NaN,F,NaN,1457,NaN,LFBD,NaN,LFBD,NaN,BCO3000,NaN,Z,NaN,A3ST,NaN,Z,NaN,FGSTB,NaN,,NaN,240124,NaN,AA55578218,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,1.0,NaN,394A61,NaN,
2064,vols_only_data_2_plus_2T,NaN,F,NaN,730,NaN,LFQQ,NaN,LFSD,NaN,SAMU59,NaN,Z,NaN,EC45,NaN,Z,NaN,FHTRP,NaN,,NaN,240124,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,
2065,vols_only_data_2_plus_2T,NaN,F,NaN,111,NaN,FZAA,NaN,LFPB,NaN,AXY2205,NaN,Z,NaN,E190,NaN,Z,NaN,9HFAB,NaN,,NaN,240124,NaN,AA55594627,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,1.0,NaN,4D2230,NaN,
2066,vols_only_data_2_plus_2T,NaN,F,NaN,1615,NaN,LFAG,NaN,LFAT,NaN,FHGET,NaN,Z,NaN,PC12,NaN,Z,NaN,FHGET,NaN,,NaN,240124,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,1.0,NaN,399893,NaN,
2067,vols_only_data_2_plus_2T,NaN,F,NaN,1645,NaN,LFAT,NaN,LFQT,NaN,FHGET,NaN,Z,NaN,PC12,NaN,Z,NaN,FHGET,NaN,,NaN,240124,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,1.0,NaN,399893,NaN,


In [2626]:
ABACUS[ABACUS["Flight identification"] == "FMY8511  "]

,Sequence number,Code,Time of departure/entry,Departure aerodrome,Arrival aerodrome,Flight identification,Main Exemption code,Type of aircraft,Operator,Aircraft Registration,Comment1,Flight date,IFPLID,Planned_aerodrome,Charging_zone_overflow,Entry_point,Exit_point,Sup_exemption_code,Source of the Aircraft Address,24-bit Aircraft Address,Comment2


In [2627]:
jour_cible[jour_cible["Flight identification"] == "FMY8511  "]

,Sequence number,Code,Time of departure/entry,Departure aerodrome,Arrival aerodrome,Flight identification,Main Exemption code,Type of aircraft,Operator,Aircraft Registration,Comment1,Flight date,IFPLID,Planned_aerodrome,Charging_zone_overflow,Entry_point,Exit_point,Sup_exemption_code,Source of the Aircraft Address,24-bit Aircraft Address,Comment2


In [2628]:
ABACUS[ABACUS["Flight identification"] == "FGNTS    "]


,Sequence number,Code,Time of departure/entry,Departure aerodrome,Arrival aerodrome,Flight identification,Main Exemption code,Type of aircraft,Operator,Aircraft Registration,Comment1,Flight date,IFPLID,Planned_aerodrome,Charging_zone_overflow,Entry_point,Exit_point,Sup_exemption_code,Source of the Aircraft Address,24-bit Aircraft Address,Comment2


In [2629]:
jour_cible[jour_cible["Flight identification"] == "FGNTS    "]

,Sequence number,Code,Time of departure/entry,Departure aerodrome,Arrival aerodrome,Flight identification,Main Exemption code,Type of aircraft,Operator,Aircraft Registration,Comment1,Flight date,IFPLID,Planned_aerodrome,Charging_zone_overflow,Entry_point,Exit_point,Sup_exemption_code,Source of the Aircraft Address,24-bit Aircraft Address,Comment2


In [2630]:
jour_2[jour_2["callSign_prevu"] == "FMY8511"]

,callSign_prevu,dep_prevu,arr_prevu,numCautra_prevu,dateRelative_prevu,typeAvion_prevu,work_prevu,heuresDep_prevu,RFL_prevu,vitesse_prevu,EOBT_prevu,regleVol_prevu,typeVol_prevu,HeurePremiereBaliseActive_prevu,IFPL_prevu,plnActive_prevu,typePlnStan,plnAnnule_prevu,dateBlock_prevu,adresseModeS_prevu,numeroPLNMprevu,flightIDprevu,baliseprevu,HeurePremiereBalise_prevu,listeBalisesprevu,indicateurprevu,carteprevu,centreTraverséprevu,listeRangPremierprevu,callSign_final,dep_final,arr_final,numCautra_final,dateRelative_final,typeAvion_final,work_final,heuresDep_final,RFL_final,vitesse_final,EOBT_final,regleVol_final,typeVol_final,HeurePremiereBaliseActive_final,IFPL_final,plnActive_final,plnAnnule_final,dateBlock_final,adresseModeS_final,HeurePremiereBalise_final,callSign_realise,dep_realise,arr_realise,numCautra_realise,dateRelative_realise,typeAvion_realise,work_realise,heuresDep_realise,RFL_realise,vitesse_realise,EOBT_realise,regleVol_realise,typeVol_realise,HeurePremiereBaliseActive_realise,IFPL_realise,plnActive_realise,plnAnnule_realise,dateBlock_realise,adresseModeS_realise,numeroPLNMrealise,flightIDrealise,baliserealise,HeurePremiereBalise_realise,listeBalisesrealise,indicateurrealise,carterealise,centreTraversérealise,listeRangPremierrealise,rangTransactiontransaction,heure,minute,accuseTrttransaction,ccrArrival,finaltransaction,isPrevu,isRealise,isFinal,work1prevu,work1final,numeroPLNMfinal,flightIDfinal,balisefinal,listeBalisesfinal,indicateurfinal,cartefinal,centreTraverséfinal,listeRangPremierfinal,work1realise,case7,case8,case9,case10,case13,case15,case16,case18,typePln,heure_de_reference,date_de_reference,utile_inutile


In [2631]:
jour_123[jour_123["Flight identification"] == "FMY8511  "]

,Sequence number,Code,Time of departure/entry,Departure aerodrome,Arrival aerodrome,Flight identification,Main Exemption code,Type of aircraft,Operator,Aircraft Registration,Comment1,Flight date,IFPLID,Planned_aerodrome,Charging_zone_overflow,Entry_point,Exit_point,Sup_exemption_code,Source of the Aircraft Address,24-bit Aircraft Address,Comment2


In [2632]:
jour_123_brut[jour_123_brut["callSign_prevu"] == "FMY8511"]

,callSign_prevu,dep_prevu,arr_prevu,numCautra_prevu,dateRelative_prevu,typeAvion_prevu,work_prevu,heuresDep_prevu,RFL_prevu,vitesse_prevu,EOBT_prevu,regleVol_prevu,typeVol_prevu,HeurePremiereBaliseActive_prevu,IFPL_prevu,plnActive_prevu,typePlnStan,plnAnnule_prevu,dateBlock_prevu,adresseModeS_prevu,numeroPLNMprevu,flightIDprevu,baliseprevu,HeurePremiereBalise_prevu,listeBalisesprevu,indicateurprevu,carteprevu,centreTraverséprevu,listeRangPremierprevu,callSign_final,dep_final,arr_final,numCautra_final,dateRelative_final,typeAvion_final,work_final,heuresDep_final,RFL_final,vitesse_final,EOBT_final,regleVol_final,typeVol_final,HeurePremiereBaliseActive_final,IFPL_final,plnActive_final,plnAnnule_final,dateBlock_final,adresseModeS_final,numeroPLNMfinal,flightIDfinal,balisefinal,HeurePremiereBalise_final,listeBalisesfinal,indicateurfinal,cartefinal,centreTraverséfinal,listeRangPremierfinal,callSign_realise,dep_realise,arr_realise,numCautra_realise,dateRelative_realise,typeAvion_realise,work_realise,heuresDep_realise,RFL_realise,vitesse_realise,EOBT_realise,regleVol_realise,typeVol_realise,HeurePremiereBaliseActive_realise,IFPL_realise,plnActive_realise,plnAnnule_realise,dateBlock_realise,adresseModeS_realise,numeroPLNMrealise,flightIDrealise,baliserealise,HeurePremiereBalise_realise,listeBalisesrealise,indicateurrealise,carterealise,centreTraversérealise,listeRangPremierrealise,rangTransactiontransaction,heure,minute,accuseTrttransaction,ccrArrival,finaltransaction,isPrevu,isRealise,isFinal,work1prevu,work1final,work1realise,case7,case8,case9,case10,case13,case15,case16,case18,typePln,heure_de_reference,date_de_reference,utile_inutile,aeronef_de_moins_de_deux_tonnes,vol_a_transmettre,vol_approche,vol_interieur,vol_frontalier,vol_VFR,code_d_exoneration,code_exploitant,compagnie_française,type_d_avion_militaire,PLN_à_verifier_TU,PLN_valide,RAZ_des_invalidites_de_type_TU,invalidite_TU,type_d_indicatif,doublon


In [2633]:
def comparaison_vol_all(vol1, vol2) -> bool:
  arrival = (vol1["Arrival aerodrome_1"] == vol2["Arrival aerodrome"])
  departure = (vol1["Departure aerodrome_1"] == vol2["Departure aerodrome"])
  id = (vol1["Flight identification_1"].rstrip() == vol2["Flight identification"])
  return (arrival and departure and id)

  #(difference_time(vol1["Time of departure/entry"], vol2["Time of departure/entry"], vol1["Flight date"], vol2["Flight date"], fichier_date1, fichier_date2) < 130)
  
def retrouver_vol_all(vol, data):
    matching_rows = data[data.apply(lambda row: comparaison_vol_all(vol, row), axis=1)]
    index = matching_rows.index
    return ([matching_rows.iloc[0]], index[0]) if len(matching_rows) >0 else ([],0)

def analyse(data_en_plus):
    data_en_plus["Raison"] = ""
    for i in range(len(data_en_plus)):
        vols_abacus_all = retrouver_vol_all(data_en_plus.iloc[i], ABACUS_toutes_origines)
        if len(vols_abacus_all[0]) > 0:
            if vols_abacus_all[0][0]["Origin of message"] != "LF":
                data_en_plus["Raison"].iloc[i] = "Origin_of_message"
            elif abs(vols_abacus_all[0][0]["Time of departure/entry"] - data_en_plus.iloc[i]["Time of departure/entry_1"]) > 100:
                data_en_plus["Raison"].iloc[i] = "Time"
    return data_en_plus

In [2634]:
data_en_plus = dataresult[dataresult["compare"] == "vols_only_data_1"]

In [2635]:
data_en_plus = analyse(data_en_plus)

In [2636]:
data_en_plus

,compare,Code_1,Code_2,Time of departure/entry_1,Time of departure/entry_2,Departure aerodrome_1,Departure aerodrome_2,Arrival aerodrome_1,Arrival aerodrome_2,Flight identification_1,Flight identification_2,Main Exemption code_1,Main Exemption code_2,Type of aircraft_1,Type of aircraft_2,Operator_1,Operator_2,Aircraft Registration_1,Aircraft Registration_2,Comment1_1,Comment1_2,Flight date_1,Flight date_2,IFPLID_1,IFPLID_2,Planned_aerodrome_1,Planned_aerodrome_2,Charging_zone_overflow_1,Charging_zone_overflow_2,Entry_point_1,Entry_point_2,Exit_point_1,Exit_point_2,Sup_exemption_code_1,Sup_exemption_code_2,Source of the Aircraft Address_1,Source of the Aircraft Address_2,24-bit Aircraft Address_1,24-bit Aircraft Address_2,Comment2_1,Comment2_2,Raison


In [2637]:
data_en_plus[['compare','Time of departure/entry_1','Raison']]

,compare,Time of departure/entry_1,Raison


In [2638]:
colonnes_a_afficher = ['compare', 'Time of departure/entry_1', 'Departure aerodrome_1', 'Arrival aerodrome_1', 'Flight identification_1', 'Type of aircraft_1', 'Raison']

data_en_plus[colonnes_a_afficher]

,compare,Time of departure/entry_1,Departure aerodrome_1,Arrival aerodrome_1,Flight identification_1,Type of aircraft_1,Raison
